# Weak-lensing galaxy shape catalogue validation

### Local calibration

Version 06/05/2021

Includes
- PSF residuals
- PSF to galaxy shapes leakage
- Additive shear bias
- Convergence E- and B-modes
- Cluster lensing

## Set-up

In [ ]:
import sys
import os
import numpy as np

import shapepipe
from shapepipe.utilities.canfar import download

In [ ]:
#from sp_validation.basic import *
from sp_validation.plot_style import *

from scipy import ndimage as ndi
from scipy import signal

from lenspack import peaks
from lenspack.peaks import find_peaks2d
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

import pandas as pd

In [ ]:
# The following commands will be replaced by import instructions, once the sp_validation scripts are stable

#sp_val = '{}/astro/repositories/github/sp_validation/sp_validation'.format(os.environ['HOME'])
sp_val = '{}/sp_validation/sp_validation'.format(os.environ['HOME'])

for sc in ['survey', 'params', 'io','basic']:
    script = os.path.join(sp_val, sc)
    %run $script

### Load parameters

### Output files and directories

In [ ]:
make_out_dirs(plot_dir, plot_subdirs, verbose=verbose)
stats_file = open_stats_file(plot_dir, stats_file_name)

## Load data

### Load merged (final) galaxy catalogue

In [ ]:
print(galaxy_cat_path)

In [ ]:
dd = np.load(galaxy_cat_path, mmap_mode=None)

#### Print some quantities to check nothing obvious is wrong with catalogue

In [ ]:
# Print all column names
print('Galaxy cat column names:')
print(dd.dtype.names)
print()

# Mean ellipticity looks reasonable, no `nan`?
print('Mean gal ellipticity =', dd['NGMIX_ELL_NOSHEAR'].mean(axis=0))

# Total number of objects
n_tot_dd = len(dd)
print('Total number of objects = {} = {:.2f} Mio'.format(n_tot_dd, n_tot_dd / 1e6))

#### Survey area and potential missing tiles
The approximate observed area is the number of tiles $\times$ 0.25 deg$^2$ (ignoring overlaps and masking).

In [ ]:
area_deg2, area_amin2, tile_IDs = get_area(dd, area_tile, verbose=verbose)

Identify missing tiles by comparing tile ID from catalogue to external input tile ID file.

In [ ]:
n_found, n_missing = missing_tiles(tile_IDs, path_tile_ID, path_missing_ID, verbose=verbose)

### Load star catalogue

In [ ]:
d_star = fits.getdata('./P3X/psf_validation_merged/psf_cat_full.fits', 1)

In [ ]:
print('Star cat column names:')
print(d_star.dtype.names)
print()

print('Mean PSF ellipticity = ({:.3g}, {:.3g})'
      ''.format(np.mean(d_star['E1_PSF_HSM']), np.mean(d_star['E2_PSF_HSM'])))

print('Number of objects = ', len(d_star))

# Data

## Matching of star catalogues
Match the star catalogue `d_star` (selected on individual exposures using size-magnitude diagram) to the stars identified on tiles by shape measurement algorithms.

This is mainly for testing, this match will not be used later.

### Match to all objects

In [ ]:
# Filter stars outside footprint for efficiency
mask_area_tiles = get_mask_footprint_P3(d_star['RA'], d_star['DEC'])

# Maximum distance in degrees for matching
thresh = 0.0002

# Match stars from exposure (PSF) catalogue to total catalogue
ind_star = match_stars2(dd['XWIN_WORLD'], dd['YWIN_WORLD'], d_star['RA'][mask_area_tiles],
                        d_star['DEC'][mask_area_tiles], thresh=thresh)
msg = 'Number of matched stars from exposures to total catalogue = {} = {:.1f}%' \
      ''.format(len(ind_star), len(ind_star) / len(d_star['RA'][mask_area_tiles])*100)
print_stats(msg, stats_file, verbose=verbose)

# Testing
ind_star_test = match_stars2(dd['XWIN_WORLD'], dd['YWIN_WORLD'], d_star['RA'], d_star['DEC'], thresh=thresh)
print('[ Check: #stars without removing tiles outside footprint = {} ]'.format(len(ind_star_test)))

# Remove stars matched to more than one target object
ind_star = np.array(list(set(ind_star)))

msg = 'Number of matched stars after removing multuple matches = {} = {:.1f}%' \
      ''.format(len(ind_star), len(ind_star) / len(d_star['RA'][mask_area_tiles])*100)
print_stats(msg, stats_file)

#### Refine: match to ngmix star sample

In [ ]:
# Flags to indicate ngmix star sample
m_star_ngmix = (dd['FLAGS'][ind_star] == 0) & \
               (dd['IMAFLAGS_ISO'][ind_star] == 0) & \
               (dd['NGMIX_MCAL_FLAGS'][ind_star] == 0) & \
               (dd['NGMIX_ELL_PSFo_NOSHEAR'][:,0][ind_star] != -10)

msg = 'Number of stars matched to ngmix star sample = {} = {:.1f}%' \
      ''.format(len(dd['FLAGS'][ind_star][m_star_ngmix]),
                len(dd['FLAGS'][ind_star][m_star_ngmix]) / len(d_star['RA'][mask_area_tiles])*100)
print_stats(msg, stats_file, verbose=verbose)

g_star_psf_ngmix = np.array([dd['NGMIX_ELL_PSFo_NOSHEAR'][:,0][ind_star][m_star_ngmix], dd['NGMIX_ELL_PSFo_NOSHEAR'][:,1][ind_star][m_star_ngmix]])
ra_star_ngmix = dd['XWIN_WORLD'][ind_star][m_star_ngmix]
dec_star_ngmix = dd['YWIN_WORLD'][ind_star][m_star_ngmix]

#### Objects with invalid PSF

In [ ]:
n_all = len(dd)

w = dd['NGMIX_ELL_PSFo_NOSHEAR'][:,0] == -10
n_inv_psf = len(np.where(w)[0])
print('Invalid PSF shear found for {}/{} = {:.1g}% objects'.format(n_inv_psf, n_all, n_inv_psf / n_all))

w = dd['NGMIX_ELL_NOSHEAR'][:,0] == -10
n_inv_gal = len(np.where(w)[0])
print('Invalid galaxy shear found for {}/{} = {:.1g}% objects'
      ''.format(n_inv_gal, n_all, n_inv_gal / n_all))

#### Star galsim

In [ ]:
msg = "Number of star matched : {}  ( {:.2f}% )".format(len(ind_star), len(ind_star)/len(d_star['RA'][mask_area_tiles])*100)
print_stats(msg, stats_file, verbose=verbose)

m_star_galsim = (dd['FLAGS'][ind_star] == 0) & (dd['IMAFLAGS_ISO'][ind_star] == 0) \
          & (dd['GALSIM_PSF_ELL_ORIGINAL_PSF'][:,0][ind_star] != -10)

msg = "Number of star after removing repetition : {}".format(len(dd['FLAGS'][ind_star][m_star_galsim]))
print_stats(msg, stats_file, verbose=verbose)

g_star_psf_galsim = np.array([dd['GALSIM_PSF_ELL_ORIGINAL_PSF'][:,0][ind_star][m_star_galsim], dd['GALSIM_PSF_ELL_ORIGINAL_PSF'][:,1][ind_star][m_star_galsim]])
ra_star_galsim = dd['XWIN_WORLD'][ind_star][m_star_galsim]
dec_star_galsim = dd['YWIN_WORLD'][ind_star][m_star_galsim]

msg = "Number of star matched : {}  ( {:.2f}% )\n".format(len(ind_star), len(ind_star)/len(d_star['RA'][mask_area_tiles])*100)
print_stats(msg, stats_file, verbose=verbose)

#### Stat stars ngmix

In [ ]:
tot_star = len(ra_star_ngmix)
tot_as_star = len(np.where(dd['SPREAD_CLASS'][ind_star][m_star_ngmix] == 0)[0])
tot_as_gal = len(np.where(dd['SPREAD_CLASS'][ind_star][m_star_ngmix] == 1)[0])
tot_as_other = len(np.where(dd['SPREAD_CLASS'][ind_star][m_star_ngmix] == 2)[0])
print("Star (single_selection) as star (spread_model) : {:.3f}%".format(tot_as_star/tot_star*100))
print("Star (single_selection) as gal (spread_model) : {:.3f}%".format(tot_as_gal/tot_star*100))
print("Star (single_selection) as other (spread_model) : {:.3f}%".format(tot_as_other/tot_star*100))
stats_file.write("Star (single_selection) as star (spread_model) : {:.3f}%\n".format(tot_as_star/tot_star*100))
stats_file.write("Star (single_selection) as gal (spread_model) : {:.3f}%\n".format(tot_as_gal/tot_star*100))
dummy = stats_file.write("Star (single_selection) as other (spread_model) : {:.3f}%\n".format(tot_as_other/tot_star*100))

#### Stat stars galsim

In [ ]:
tot_star = len(ra_star_galsim)
tot_as_star = len(np.where(dd['SPREAD_CLASS'][ind_star][m_star_galsim] == 0)[0])
tot_as_gal = len(np.where(dd['SPREAD_CLASS'][ind_star][m_star_galsim] == 1)[0])
tot_as_other = len(np.where(dd['SPREAD_CLASS'][ind_star][m_star_galsim] == 2)[0])
print("Star (single_selection) as star (spread_model) : {:.3f}%".format(tot_as_star/tot_star*100))
print("Star (single_selection) as gal (spread_model) : {:.3f}%".format(tot_as_gal/tot_star*100))
print("Star (single_selection) as other (spread_model) : {:.3f}%".format(tot_as_other/tot_star*100))
stats_file.write("Star (single_selection) as star (spread_model) : {:.3f}%\n".format(tot_as_star/tot_star*100))
stats_file.write("Star (single_selection) as gal (spread_model) : {:.3f}%\n".format(tot_as_gal/tot_star*100))
dummy = stats_file.write("Star (single_selection) as other (spread_model) : {:.3f}%\n".format(tot_as_other/tot_star*100))

### Select galaxies

#### Galxies ngmix

In [ ]:
sm_classif = dd['SPREAD_MODEL']+2*dd['SPREADERR_MODEL']
m_gal_ngmix = \
    (sm_classif > 0.0035) \
    & (dd['SPREAD_MODEL'] > 0.) \
    & (dd['SPREAD_MODEL'] < 0.03) \
    & (dd['MAG_AUTO'] < 26) \
    & (dd['MAG_AUTO'] > 20) \
    & (dd['FLAGS'] == 0) \
    & (dd['IMAFLAGS_ISO'] == 0) \
    & (dd['NGMIX_MCAL_FLAGS'] == 0) \
    & (dd['NGMIX_ELL_PSFo_NOSHEAR'][:,0] != -10) \
    & (dd['NGMIX_MOM_FAIL'] == 0) \
    & (dd['N_EPOCH'] > 0) \
    & (dd['NGMIX_N_EPOCH'] > 0)
print('Objects selected as galaxies = {}'.format(len(np.where(m_gal_ngmix)[0])))

#### Galaxies galsim

In [ ]:
sm_classif = dd['SPREAD_MODEL']+2*dd['SPREADERR_MODEL']
#m_gal = (dd['SPREAD_CLASS'] == 1) & (dd['SPREAD_MODEL'] > 0.)  (dd['MAG_AUTO'] > 20)   ((dd['FLAGS'] == 0) | (dd['FLAGS'] == 2))
m_gal_galsim = (sm_classif > 0.0035) & (dd['SPREAD_MODEL'] > 0.) & (dd['SPREAD_MODEL'] < 0.03) \
        & (dd['MAG_AUTO'] < 26) & (dd['MAG_AUTO'] > 20) \
        & (dd['FLAGS'] == 0) & (dd['IMAFLAGS_ISO'] == 0) \
        & (dd['GALSIM_PSF_ELL_ORIGINAL_PSF'][:,0] != -10) \
        & (dd['N_EPOCH'] > 0)

### Compile metacal for galaxies  

Masking : Flags == 0   &   $\frac{T_{gal}}{T_{psf}}$ > 0.5   &   SNR > 10   &   SNR < 500

#### Ngmix

Global calibration

In [ ]:
gal_metacal_ngmix = metacal(dd, m_gal_ngmix)

In [ ]:
g_ngmix = np.array([gal_metacal_ngmix.ns['g1'][gal_metacal_ngmix.mask_dict['ns']], gal_metacal_ngmix.ns['g2'][gal_metacal_ngmix.mask_dict['ns']]])
g_corr_ngmix = np.linalg.inv(gal_metacal_ngmix.R).dot(g_ngmix)
w_ngmix = gal_metacal_ngmix.ns['w'][gal_metacal_ngmix.mask_dict['ns']]
ra_ngmix = dd['XWIN_WORLD'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]
dec_ngmix = dd['YWIN_WORLD'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]
mag_ngmix = dd['MAG_AUTO'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]
snr_ngmix = dd['NGMIX_FLUX_NOSHEAR'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]/dd['NGMIX_FLUX_ERR_NOSHEAR'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]

In [ ]:
print(np.sqrt(np.var(g_ngmix[0])+np.var(g_ngmix[1])))
print(np.sqrt(R_shear_moy[0,0]*(np.var(g_corr_ngmix[0])+np.var(g_corr_ngmix[1]))))

In [ ]:
np.save('./P3X/var_save/mag_ngmix.npy',mag_ngmix)
np.save('./P3X/var_save/snr_ngmix.npy',snr_ngmix)

In [ ]:
ra_ngmix = np.load('./P3X/var_save/ra_ngmix.npy')
dec_ngmix = np.load('./P3X/var_save/dec_ngmix.npy')
w_ngmix = np.load('./P3X/var_save/w_ngmix.npy')


In [ ]:
# R matrix and std needed
R_shear_moy = np.mean(gal_metacal_ngmix.R_shear,2)
R_shear_moy_std = gal_metacal_ngmix.R_shear_std
R_selec_moy = gal_metacal_ngmix.R_selection
R_selec_moy_std = gal_metacal_ngmix.R_selection_std
R_tot_moy = gal_metacal_ngmix.R

In [ ]:
R_shear_moy = np.load('./P3X/var_save/R_shear_moy.npy')
R_shear_moy_std = np.load('./P3X/var_save/R_shear_moy_std.npy')
R_selec_moy = np.load('./P3X/var_save/R_selec_moy.npy')
R_selec_moy_std  = np.load('./P3X/var_save/R_selec_moy_std.npy')
R_tot_moy = np.load('./P3X/var_save/R_tot_moy.npy')

In [ ]:
# Quantity for correlation matrix
# PSF
Psf1 = dd['NGMIX_ELL_PSFo_NOSHEAR'][:,0][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]
Psf2 = dd['NGMIX_ELL_PSFo_NOSHEAR'][:,1][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]
module_psf = np.sqrt(Psf1**2 + Psf2**2)

# Size gal
size_gal = dd['NGMIX_T_NOSHEAR'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]

# N epoch
nepoch = dd['NGMIX_N_EPOCH'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]

# Mag
mag = dd['MAG_AUTO'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]

# snr
snr_win = dd['SNR_WIN'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]

# taille psf
t_psf = dd['NGMIX_Tpsf_NOSHEAR'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]

#### Local calibration

In [ ]:
# Projection (rad,dec) -> (x,y)
x, y = radec2xy(np.mean(dd['XWIN_WORLD']), np.mean(dd['YWIN_WORLD']), dd['XWIN_WORLD'], dd['YWIN_WORLD'])

In [ ]:
# Define mix, max and size
min_x = np.min(x)
max_x = np.max(x)
min_y = np.min(y)
max_y = np.max(y)

size_x = max_x - min_x
size_y = max_y - min_y

In [ ]:
print('size ra in deg',np.rad2deg(size_x))
print('size dec in deg',np.rad2deg(size_y))

In [ ]:
# W3
#extend_x = 12-8.614568497714435
#extend_y = 8-7.192988571604766

#P3
extend_x = 36-34.643958374949605
extend_y = 20-17.71186344852433

In [ ]:
# Extend
max_x = max_x + np.deg2rad(extend_x)
max_y = max_y + np.deg2rad(extend_y)
size_x = max_x - min_x 
size_y = max_y - min_y

# Check that new size is ok
print(np.rad2deg(size_x))
print(np.rad2deg(size_y))

In [ ]:
size_deg_x = 36 #P3
size_deg_y = 20

#size_deg_x = 12 #W3
#size_deg_y = 8

reso = 0.4

In [ ]:
# Size in pixel
Nx = int(size_deg_x*60/reso)
Ny = int(size_deg_y*60/reso)
print(Nx,Ny)

In [ ]:
# Project (ra,dec) -> (x,y)
xx, yy = radec2xy(np.mean(ra_ngmix), np.mean(dec_ngmix), ra_ngmix, dec_ngmix)

In [ ]:
xx = np.load('./P3X/var_save/xx.npy')
yy = np.load('./P3X/var_save/yy.npy')

In [ ]:
# Additive bias
c1_ngmix, err_c1_ngmix = jackknif_weighted_average(g_corr_ngmix[0], w_ngmix, remove_size=0.05, n_realization=500)
c2_ngmix, err_c2_ngmix = jackknif_weighted_average(g_corr_ngmix[1], w_ngmix, remove_size=0.05, n_realization=500)

In [ ]:
c1_ngmix = np.load('./P3X/var_save/c1_ngmix.npy')
err_c1_ngmix = np.load('./P3X/var_save/err_c1_ngmix.npy')
c2_ngmix = np.load('./P3X/var_save/c2_ngmix.npy')
err_c2_ngmix = np.load('./P3X/var_save/err_c2_ngmix.npy')

In [ ]:
def local_calib(npix_x,npix_y,dm,sigm):
    g1_final = np.array([])
    g2_final = np.array([])
    g1_final_dm = np.array([])
    g2_final_dm = np.array([])
    R_selec = np.zeros((2,2,npix_y,npix_x))
    R_shear = np.zeros((2,2,npix_y,npix_x))
    ra_ngmix2 = np.array([])
    dec_ngmix2 = np.array([])
    R_selec_std = np.zeros((2,2,npix_y,npix_x))
    R_shear_std = np.zeros((2,2,npix_y,npix_x))
    c1_ngmix_cut = np.zeros((npix_y,npix_x))
    c2_ngmix_cut = np.zeros((npix_y,npix_x))
    err_c1_ngmix_cut = np.zeros((npix_y,npix_x))
    err_c2_ngmix_cut = np.zeros((npix_y,npix_x))
    
    ngal = bin2d(xx, yy, npix=(npix_x,npix_y),extent=(min_x,max_x,min_y,max_y))

    # Cut the catalogue into subpatches -> dd_cut
    for i in range(npix_x):
        for j in range(npix_y):
            if(i==npix_x and j!=npix_y):
                dd_cut = dd[np.where((x >= min_x + (i*size_x/npix_x)) & (x <= max_x) \
                                     & (y >= min_y + (j*size_y/npix_y)) & (y < min_y + ((j+1)*size_y/npix_y)))]
            elif(j==npix_x and i!=npix_y):
                dd_cut = dd[np.where((x >= min_x + (i*size_x/npix_x)) & (x < min_x + ((i+1)*size_x/npix_x)) \
                                     & (y >= min_y + (j*size_y/npix_y)) & (y <= max_y))] 
            elif(j==npix_x and i==npix_y):
                dd_cut = dd[np.where((x >= min_x + (i*size_x/npix_x)) & (x <= max_x) \
                                     & (y >= min_y + (j*size_y/npix_y)) & (y <= max_y ))]
            else:
                dd_cut = dd[np.where((x >= min_x + (i*size_x/npix_x)) & (x < min_x + ((i+1)*size_x/npix_x)) \
                                     & (y >= min_y + (j*size_y/npix_y)) & (y < min_y + ((j+1)*size_y/npix_y)))]
        
            # Mask of dd_cut
            sm_classif_cut = dd_cut['SPREAD_MODEL']+2*dd_cut['SPREADERR_MODEL']
            m_gal_ngmix_cut = \
                (sm_classif_cut > 0.0035) \
                & (dd_cut['SPREAD_MODEL'] > 0.) \
                & (dd_cut['SPREAD_MODEL'] < 0.03) \
                & (dd_cut['MAG_AUTO'] < 26) \
                & (dd_cut['MAG_AUTO'] > 20) \
                & (dd_cut['FLAGS'] == 0) \
                & (dd_cut['IMAFLAGS_ISO'] == 0) \
                & (dd_cut['NGMIX_MCAL_FLAGS'] == 0) \
                & (dd_cut['NGMIX_ELL_PSFo_NOSHEAR'][:,0] != -10) \
                & (dd_cut['NGMIX_MOM_FAIL'] == 0) \
                & (dd_cut['N_EPOCH'] > 0) \
                & (dd_cut['NGMIX_N_EPOCH'] > 0)
            print('Objects selected as galaxies = {}'.format(len(np.where(m_gal_ngmix_cut)[0])))

            # Apply metacal to dd_cut
            gal_metacal_ngmix_cut = metacal(dd_cut, m_gal_ngmix_cut)
        
            # Save ra & dec to keep the order        
            ra_ngmix_temp = dd_cut['XWIN_WORLD'][m_gal_ngmix_cut][gal_metacal_ngmix_cut.mask_dict['ns']]
            ra_ngmix2 = np.concatenate((ra_ngmix2,ra_ngmix_temp))
            dec_ngmix_temp = dd_cut['YWIN_WORLD'][m_gal_ngmix_cut][gal_metacal_ngmix_cut.mask_dict['ns']]
            dec_ngmix2 = np.concatenate((dec_ngmix2,dec_ngmix_temp))
            
            w_ngmix_cut = gal_metacal_ngmix_cut.ns['w'][gal_metacal_ngmix_cut.mask_dict['ns']]
        
            g_ngmix_cut = np.array([gal_metacal_ngmix_cut.ns['g1'][gal_metacal_ngmix_cut.mask_dict['ns']], gal_metacal_ngmix_cut.ns['g2'][gal_metacal_ngmix_cut.mask_dict['ns']]])
        
            # If numver of galaxy is ok, metacal local
            if (ngal[j,i] > (np.mean(ngal))/2):
                g_corr_ngmix_cut = np.linalg.inv(gal_metacal_ngmix_cut.R).dot(g_ngmix_cut)
                
                # Add of delta m 
                R_dm = gal_metacal_ngmix_cut.R + np.ones((2,2))*(dm + np.random.normal(0, sigm))
                g_corr_ngmix_cut_dm = np.linalg.inv(R_dm).dot(g_ngmix_cut)
                
                # Additive bias
                c1_ngmix_cut[j,i], err_c1_ngmix_cut[j,i] = jackknif_weighted_average2(g_corr_ngmix_cut[0], w_ngmix_cut, remove_size=0.05, n_realization=500)
                c2_ngmix_cut[j,i], err_c2_ngmix_cut[j,i] = jackknif_weighted_average2(g_corr_ngmix_cut[1], w_ngmix_cut, remove_size=0.05, n_realization=500)
                
                # Save R matrix and std
                R_selec[:,:,j,i] = gal_metacal_ngmix_cut.R_selection
                R_shear[:,:,j,i] = np.mean(gal_metacal_ngmix_cut.R_shear,2)
                R_selec_std[:,:,j,i] = gal_metacal_ngmix_cut.R_selection_std
                R_shear_std[:,:,j,i] = gal_metacal_ngmix_cut.R_shear_std
                                
            # If low number of galaxy, we use value of globaal metacal
            else:
                g_corr_ngmix_cut = np.linalg.inv(R_tot_moy).dot(g_ngmix_cut)
                R_dm = R_tot_moy + np.ones((2,2))*(dm + np.random.normal(0, sigm))
                g_corr_ngmix_cut_dm = np.linalg.inv(R_dm).dot(g_ngmix_cut)
                
                c1_ngmix_cut[j,i] = c1_ngmix
                err_c1_ngmix_cut[j,i] = err_c1_ngmix
                c2_ngmix_cut[j,i] = c2_ngmix
                err_c2_ngmix_cut[j,i] = err_c2_ngmix
            
                R_selec[:,:,j,i] = R_selec_moy
                R_shear[:,:,j,i] = R_shear_moy
                R_selec_std[:,:,j,i] = R_selec_moy_std
                R_shear_std[:,:,j,i] = R_shear_moy_std
                             
            g1_final = np.concatenate((g1_final, g_corr_ngmix_cut[0]))  
            g2_final = np.concatenate((g2_final, g_corr_ngmix_cut[1]))
            g1_final_dm = np.concatenate((g1_final_dm, g_corr_ngmix_cut_dm[0]))  
            g2_final_dm = np.concatenate((g2_final_dm, g_corr_ngmix_cut_dm[1]))
            
    return g1_final, g2_final, g1_final_dm, g2_final_dm, R_selec, R_shear, ra_ngmix2, dec_ngmix2, R_selec_std, R_shear_std,c1_ngmix_cut,c2_ngmix_cut,err_c1_ngmix_cut,err_c2_ngmix_cut

In [ ]:
# Apply local metacal, with different cutting size : 1 px = 0.25, 0.5, 1, 2, 4 square degree

With dm = 0.007 (Axel's simulations)

In [ ]:
g1_final_05deg, g2_final_05deg, g1_final_dm_05deg, g2_final_dm_05deg, R_selec_05deg, R_shear_05deg, ra_ngmix2_05deg, dec_ngmix2_05deg, R_selec_std_05deg, R_shear_std_05deg,c1_ngmix_05deg,c2_ngmix_05deg,err_c1_ngmix_05deg,err_c2_ngmix_05deg = local_calib(size_deg_x*2,size_deg_y*2,0.007,0)

In [ ]:
g1_final_025deg, g2_final_025deg, g1_final_dm_025deg, g2_final_dm_025deg, R_selec_025deg, R_shear_025deg, ra_ngmix2_025deg, dec_ngmix2_025deg, R_selec_std_025deg, R_shear_std_025deg,c1_ngmix_025deg,c2_ngmix_025deg,err_c1_ngmix_025deg,err_c2_ngmix_025deg = local_calib(size_deg_x*4,size_deg_y*4,0.007,0)

In [ ]:
g1_final_1deg, g2_final_1deg, g1_final_dm_1deg, g2_final_dm_1deg, R_selec_1deg, R_shear_1deg, ra_ngmix2_1deg, dec_ngmix2_1deg, R_selec_std_1deg, R_shear_std_1deg,c1_ngmix_1deg,c2_ngmix_1deg,err_c1_ngmix_1deg,err_c2_ngmix_1deg = local_calib(size_deg_x,size_deg_y,0.007,0)

In [ ]:
g1_final_2deg, g2_final_2deg, g1_final_dm_2deg, g2_final_dm_2deg, R_selec_2deg, R_shear_2deg, ra_ngmix2_2deg, dec_ngmix2_2deg, R_selec_std_2deg, R_shear_std_2deg,c1_ngmix_2deg,c2_ngmix_2deg,err_c1_ngmix_2deg,err_c2_ngmix_2deg = local_calib(int(size_deg_x/2),int(size_deg_y/2),0.007,0)

In [ ]:
g1_final_4deg, g2_final_4deg, g1_final_dm_4deg, g2_final_dm_4deg, R_selec_4deg, R_shear_4deg, ra_ngmix2_4deg, dec_ngmix2_4deg, R_selec_std_4deg, R_shear_std_4deg,c1_ngmix_4deg,c2_ngmix_4deg,err_c1_ngmix_4deg,err_c2_ngmix_4deg = local_calib(int(size_deg_x/4),int(size_deg_y/4),0.007,0)

In [ ]:
# Load some previous data saved for plot, correlation matrix,...
c1_ngmix_1deg = np.load('./P3X/var_save/dm_0_013/c1_ngmix_1deg.npy')
c2_ngmix_1deg = np.load('./P3X/var_save/dm_0_013/c2_ngmix_1deg.npy')
err_c1_ngmix_1deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_1deg.npy')
err_c2_ngmix_1deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_1deg.npy')
c1_ngmix_2deg = np.load('./P3X/var_save/dm_0_013/c1_ngmix_2deg.npy')
c2_ngmix_2deg = np.load('./P3X/var_save/dm_0_013/c2_ngmix_2deg.npy')
err_c1_ngmix_2deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_2deg.npy')
err_c2_ngmix_2deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_2deg.npy')
c1_ngmix_4deg = np.load('./P3X/var_save/dm_0_013/c1_ngmix_4deg.npy')
c2_ngmix_4deg = np.load('./P3X/var_save/dm_0_013/c2_ngmix_4deg.npy')
err_c1_ngmix_4deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_4deg.npy')
err_c2_ngmix_4deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_4deg.npy')
c1_ngmix_05deg = np.load('./P3X/var_save/dm_0_013/c1_ngmix_05deg.npy')
c2_ngmix_05deg = np.load('./P3X/var_save/dm_0_013/c2_ngmix_05deg.npy')
err_c1_ngmix_05deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_05deg.npy')
err_c2_ngmix_05deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_05deg.npy')
c1_ngmix_025deg = np.load('./P3X/var_save/dm_0_013/c1_ngmix_025deg.npy')
c2_ngmix_025deg = np.load('./P3X/var_save/dm_0_013/c2_ngmix_025deg.npy')
err_c1_ngmix_025deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_025deg.npy')
err_c2_ngmix_025deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_025deg.npy')

c1_ngmix = np.load('./P3X/var_save/c1_ngmix.npy')
c2_ngmix = np.load('./P3X/var_save/c2_ngmix.npy')

R_selec_1deg = np.load('./P3X/var_save/dm_0_013/R_selec_1deg.npy')
R_selec_2deg = np.load('./P3X/var_save/dm_0_013/R_selec_2deg.npy')
R_selec_4deg = np.load('./P3X/var_save/dm_0_013/R_selec_4deg.npy')
R_selec_05deg = np.load('./P3X/var_save/dm_0_013/R_selec_05deg.npy')
R_selec_025deg = np.load('./P3X/var_save/dm_0_013/R_selec_025deg.npy')
R_shear_1deg = np.load('./P3X/var_save/dm_0_013/R_shear_1deg.npy')
R_shear_2deg = np.load('./P3X/var_save/dm_0_013/R_shear_2deg.npy')
R_shear_4deg = np.load('./P3X/var_save/dm_0_013/R_shear_4deg.npy')
R_shear_05deg = np.load('./P3X/var_save/dm_0_013/R_shear_05deg.npy')
R_shear_025deg = np.load('./P3X/var_save/dm_0_013/R_shear_025deg.npy')

In [ ]:
plt.figure(figsize=(74,45))
plt.subplot(5,4,1)
ax = plt.gca()
im = ax.imshow(c1_ngmix_025deg)
plt.title(r'$c_1$',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,2)
ax = plt.gca()
im = ax.imshow(err_c1_ngmix_025deg)
plt.title(r'err $c_1$',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,3)
ax = plt.gca()
im = ax.imshow(c2_ngmix_025deg)
plt.title(r'$c_2$',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,4)
ax = plt.gca()
im = ax.imshow(err_c2_ngmix_025deg)
plt.title(r'err $c_2$',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,5)
ax = plt.gca()
im = ax.imshow(c1_ngmix_05deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,6)
ax = plt.gca()
im = ax.imshow(err_c1_ngmix_05deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,7)
ax = plt.gca()
im = ax.imshow(c2_ngmix_05deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,8)
ax = plt.gca()
im = ax.imshow(err_c2_ngmix_05deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,9)
ax = plt.gca()
im = ax.imshow(c1_ngmix_1deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,10)
ax = plt.gca()
im = ax.imshow(err_c1_ngmix_1deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,11)
ax = plt.gca()
im = ax.imshow(c2_ngmix_1deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,12)
ax = plt.gca()
im = ax.imshow(err_c2_ngmix_1deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,13)
ax = plt.gca()
im = ax.imshow(c1_ngmix_2deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,14)
ax = plt.gca()
im = ax.imshow(err_c1_ngmix_2deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,15)
ax = plt.gca()
im = ax.imshow(c2_ngmix_2deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,16)
ax = plt.gca()
im = ax.imshow(err_c2_ngmix_2deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,17)
ax = plt.gca()
im = ax.imshow(c1_ngmix_4deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,18)
ax = plt.gca()
im = ax.imshow(err_c1_ngmix_4deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,19)
ax = plt.gca()
im = ax.imshow(c2_ngmix_4deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,20)
ax = plt.gca()
im = ax.imshow(err_c2_ngmix_4deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)
plt.savefig('./P3X/c.png',bbox_inches = 'tight',pad_inches = 0)
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.hist(np.ravel(c1_ngmix_025deg),label=r'$0.25^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(c1_ngmix_05deg),label='1 px = 0.5° x 0.5°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(c1_ngmix_1deg),label=r'$1^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(c1_ngmix_2deg),label='1 px = 2° x 2°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(c1_ngmix_4deg),label=r'$4^\circ$',histtype='step',density=True,bins=50)
plt.axvline(x=c1_ngmix,c='k',label='global calibration')
plt.legend(fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(-0.02,0.02)
plt.xlabel(r'$c_1$', fontsize=30)
plt.savefig('./W3/c1_hist.png',bbox_inches = 'tight',pad_inches = 0.1)
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.hist(np.ravel(c2_ngmix_025deg),label=r'$0.25^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(c2_ngmix_05deg),label='1 px = 0.5° x 0.5°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(c2_ngmix_1deg),label=r'$1^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(c2_ngmix_2deg),label='1 px = 2° x 2°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(c2_ngmix_4deg),label=r'$4^\circ$',histtype='step',density=True,bins=50)
plt.axvline(x=c2_ngmix,c='k',label='global calibration')
plt.legend(fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(-0.02,0.02)
plt.xlabel(r'$c_2$', fontsize=30)
plt.savefig('./W3/c2_hist.png',bbox_inches = 'tight',pad_inches = 0.1)
plt.show()

In [ ]:
# Binning e with global calibration
e1map, e2map = bin2d(xx, yy, npix=(Nx,Ny), v=(g_corr_ngmix[0],g_corr_ngmix[1]),extent=(min_x,max_x,min_y,max_y))
emap = np.array([e1map,e2map])

In [ ]:
# Binning local calibration
# Calib with 0.5 deg
xx_025deg, yy_025deg = radec2xy(np.mean(ra_ngmix2_025deg), np.mean(dec_ngmix2_025deg), ra_ngmix2_025deg, dec_ngmix2_025deg)

g1map_025deg, g2map_025deg = bin2d(xx_025deg, yy_025deg, npix=(Nx,Ny), v=(g1_final_025deg,g2_final_025deg),extent=(min_x,max_x,min_y,max_y))
gmap_025deg = np.array([g1map_025deg,g2map_025deg])

g1map_dm_025deg, g2map_dm_025deg = bin2d(xx_025deg, yy_025deg, npix=(Nx,Ny), v=(g1_final_dm_025deg,g2_final_dm_025deg),extent=(min_x,max_x,min_y,max_y))
gmap_dm_025deg = np.array([g1map_dm_025deg,g2map_dm_025deg])

# Calib with 0.5 deg
xx_05deg, yy_05deg = radec2xy(np.mean(ra_ngmix2_05deg), np.mean(dec_ngmix2_05deg), ra_ngmix2_05deg, dec_ngmix2_05deg)

g1map_05deg, g2map_05deg = bin2d(xx_05deg, yy_05deg, npix=(Nx,Ny), v=(g1_final_05deg,g2_final_05deg),extent=(min_x,max_x,min_y,max_y))
gmap_05deg = np.array([g1map_05deg,g2map_05deg])

g1map_dm_05deg, g2map_dm_05deg = bin2d(xx_05deg, yy_05deg, npix=(Nx,Ny), v=(g1_final_dm_05deg,g2_final_dm_05deg),extent=(min_x,max_x,min_y,max_y))
gmap_dm_05deg = np.array([g1map_dm_05deg,g2map_dm_05deg])

# Calib with 1 deg
xx_1deg, yy_1deg = radec2xy(np.mean(ra_ngmix2_1deg), np.mean(dec_ngmix2_1deg), ra_ngmix2_1deg, dec_ngmix2_1deg)

g1map_1deg, g2map_1deg = bin2d(xx_1deg, yy_1deg, npix=(Nx,Ny), v=(g1_final_1deg,g2_final_1deg),extent=(min_x,max_x,min_y,max_y))
gmap_1deg = np.array([g1map_1deg,g2map_1deg])

g1map_dm_1deg, g2map_dm_1deg = bin2d(xx_1deg, yy_1deg, npix=(Nx,Ny), v=(g1_final_dm_1deg,g2_final_dm_1deg),extent=(min_x,max_x,min_y,max_y))
gmap_dm_1deg = np.array([g1map_dm_1deg,g2map_dm_1deg])

# Calib with 2 deg
xx_2deg, yy_2deg = radec2xy(np.mean(ra_ngmix2_2deg), np.mean(dec_ngmix2_2deg), ra_ngmix2_2deg, dec_ngmix2_2deg)

g1map_2deg, g2map_2deg = bin2d(xx_2deg, yy_2deg, npix=(Nx,Ny), v=(g1_final_2deg,g2_final_2deg),extent=(min_x,max_x,min_y,max_y))
gmap_2deg = np.array([g1map_2deg,g2map_2deg])

g1map_dm_2deg, g2map_dm_2deg = bin2d(xx_2deg, yy_2deg, npix=(Nx,Ny), v=(g1_final_dm_2deg,g2_final_dm_2deg),extent=(min_x,max_x,min_y,max_y))
gmap_dm_2deg = np.array([g1map_dm_2deg,g2map_dm_2deg])

# Calib with 4 deg
xx_4deg, yy_4deg = radec2xy(np.mean(ra_ngmix2_4deg), np.mean(dec_ngmix2_4deg), ra_ngmix2_4deg, dec_ngmix2_4deg)

g1map_4deg, g2map_4deg = bin2d(xx_4deg, yy_4deg, npix=(Nx,Ny), v=(g1_final_4deg,g2_final_4deg),extent=(min_x,max_x,min_y,max_y))
gmap_4deg = np.array([g1map_4deg,g2map_4deg])

g1map_dm_4deg, g2map_dm_4deg = bin2d(xx_4deg, yy_4deg, npix=(Nx,Ny), v=(g1_final_dm_4deg,g2_final_dm_4deg),extent=(min_x,max_x,min_y,max_y))
gmap_dm_4deg = np.array([g1map_dm_4deg,g2map_dm_4deg])

In [ ]:
emap = np.load('./P3X/var_save/emap.npy')
gmap_025deg = np.load('./P3X/var_save/dm_0_013/gmap_025deg.npy')
gmap_05deg = np.load('./P3X/var_save/dm_0_013/gmap_05deg.npy')
gmap_1deg = np.load('./P3X/var_save/dm_0_013/gmap_1deg.npy')
gmap_2deg = np.load('./P3X/var_save/dm_0_013/gmap_2deg.npy')
gmap_4deg = np.load('./P3X/var_save/dm_0_013/gmap_4deg.npy')

In [ ]:
# Quantity for correlation : binning
psf1map, psf2map = bin2d(xx, yy, npix=(Nx,Ny), v=(Psf1,Psf2),extent=(min_x,max_x,min_y,max_y))
psf = np.array([psf1map,psf2map])

module_psf_map, size_gal_map, nepoch_map, mag_map, snr_map, snr_ngmix_map, tpsf_map, w_ngmix_map = bin2d(xx, yy, npix=(Nx,Ny), v=(module_psf,size_gal,nepoch,mag,snr_win,snr_ngmix,t_psf,w_ngmix),extent=(min_x,max_x,min_y,max_y))

In [ ]:
snr_ngmix_map = bin2d(xx, yy, npix=(Nx,Ny), v=(snr_ngmix),extent=(min_x,max_x,min_y,max_y))

In [ ]:
module_psf_map = np.load('./P3X/var_save/module_psf_map.npy')
psf1map = np.load('./P3X/var_save/psf1map.npy')
psf2map = np.load('./P3X/var_save/psf2map.npy')
size_gal_map = np.load('./P3X/var_save/size_gal_map.npy')
nepoch_map = np.load('./P3X/var_save/nepoch_map.npy')
mag_map = np.load('./P3X/var_save/mag_map.npy')
snr_map = np.load('./P3X/var_save/snr_map.npy')
tpsf_map = np.load('./P3X/var_save/tpsf_map.npy')
w_ngmix_map = np.load('./P3X/var_save/w_ngmix_map.npy')
snr_ngmix_map = np.load('./P3X/var_save/snr_ngmix_map.npy')

In [ ]:
def compute_R_real_std(npix_x,npix_y,factor,R):
    # Compute std of R with jackknife spatial
    
    R_real_std = np.zeros((2,2,npix_y,npix_x))

    for i in range(npix_y):
        for j in range(npix_x):
            if (np.sum(R[0,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[0,0,i,j] = jackknif_weighted_average(np.ravel(R[0,0,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[0,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[0,0,i,j] = np.nan
                
            if (np.sum(R[1,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[1,1,i,j] = jackknif_weighted_average(np.ravel(R[1,1,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[1,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[1,1,i,j] = np.nan
                
            if (np.sum(R[0,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[0,1,i,j] = jackknif_weighted_average(np.ravel(R[0,1,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[0,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[0,1,i,j] = np.nan
                
            if (np.sum(R[1,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[1,0,i,j] = jackknif_weighted_average(np.ravel(R[1,0,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[1,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[1,0,i,j] = np.nan
                
    return R_real_std

In [ ]:
# Std of R selec 1 deg thanks to 2x2 px of R selec 0.5 deg
R_selec_real_std_1deg = compute_R_real_std(size_deg_x,size_deg_y,2,R_selec_05deg)

# Std of R selec 2 deg thanks to 2x2 px of R selec 1 deg
R_selec_real_std_2deg = compute_R_real_std(int(size_deg_x/2),int(size_deg_y/2),2,R_selec_1deg)

# Std of R selec 4 deg thanks to 4x4 px of R selec 1 deg
R_selec_real_std_4deg = compute_R_real_std(int(size_deg_x/4),int(size_deg_y/4),4,R_selec_1deg)

In [ ]:
# Std of R selec 2 deg thanks to 4x4 px of R selec 0.5 deg
R_selec_real_std_2deg_from05 = compute_R_real_std(int(size_deg_x/2),int(size_deg_y/2),4,R_selec_05deg)

# Std of R selec 4 deg thanks to 8x8 px of R selec 0.5 deg
R_selec_real_std_4deg_from05 = compute_R_real_std(int(size_deg_x/4),int(size_deg_y/4),8,R_selec_05deg)

In [ ]:
# Std of R selec 05deg thanks to 2x2 px of R selec 0.25 deg
R_selec_real_std_05deg_from025 = compute_R_real_std(size_deg_x*2,size_deg_y*2,2,R_selec_025deg)

In [ ]:
# n gal needed
#ngal_00001deg = bin2d(xx,yy,npix=(size_deg_x*10000,size_deg_y*10000),extent=(min_x,max_x,min_y,max_y))
ngal_0001deg = bin2d(xx,yy,npix=(size_deg_x*1000,size_deg_y*1000),extent=(min_x,max_x,min_y,max_y))
ngal_001deg = bin2d(xx,yy,npix=(size_deg_x*100,size_deg_y*100),extent=(min_x,max_x,min_y,max_y))
ngal_01deg = bin2d(xx,yy,npix=(size_deg_x*10,size_deg_y*10),extent=(min_x,max_x,min_y,max_y))
ngal_05deg = bin2d(xx,yy,npix=(size_deg_x*2,size_deg_y*2),extent=(min_x,max_x,min_y,max_y))
ngal_025deg = bin2d(xx,yy,npix=(size_deg_x*4,size_deg_y*4),extent=(min_x,max_x,min_y,max_y))
ngal_1deg = bin2d(xx,yy,npix=(size_deg_x,size_deg_y),extent=(min_x,max_x,min_y,max_y))
ngal_2deg = bin2d(xx,yy,npix=(int(size_deg_x/2),int(size_deg_y/2)),extent=(min_x,max_x,min_y,max_y))
ngal_4deg = bin2d(xx,yy,npix=(int(size_deg_x/4),int(size_deg_y/4)),extent=(min_x,max_x,min_y,max_y))

In [ ]:
#ngal_0001deg = np.load('./P3X/var_save/ngal_0001deg.npy')
#ngal_001deg = np.load('./P3X/var_save/ngal_001deg.npy')
#ngal_01deg = np.load('./P3X/var_save/ngal_01deg.npy')
ngal_05deg = np.load('./P3X/var_save/ngal_05deg.npy')
ngal_025deg = np.load('./P3X/var_save/ngal_025deg.npy')
ngal_1deg = np.load('./P3X/var_save/ngal_1deg.npy')
ngal_2deg = np.load('./P3X/var_save/ngal_2deg.npy')
ngal_4deg = np.load('./P3X/var_save/ngal_4deg.npy')

In [ ]:
# Plot R_11 selec, its std, ngal
plt.figure(figsize=(55,45))
plt.subplot(5,3,1)
ax = plt.gca()
im = ax.imshow(R_selec_025deg[0,0,:,:])
plt.title(r'$R_{selection}$',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,2)
ax = plt.gca()
plt.title(r'std $R_{selection}$',fontsize=50)
plt.axis('off')

plt.subplot(5,3,3)
ax = plt.gca()
im = ax.imshow(ngal_025deg)
plt.title(r'n gal',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,4)
ax = plt.gca()
im = ax.imshow(R_selec_05deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,5)
ax = plt.gca()
im = ax.imshow(R_selec_real_std_05deg_from025[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,6)
ax = plt.gca()
im = ax.imshow(ngal_05deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,7)
ax = plt.gca()
im = ax.imshow(R_selec_1deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,8)
ax = plt.gca()
im = ax.imshow(R_selec_real_std_1deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,9)
ax = plt.gca()
im = ax.imshow(ngal_1deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,10)
ax = plt.gca()
im = ax.imshow(R_selec_2deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,11)
ax = plt.gca()
im = ax.imshow(R_selec_real_std_2deg_from05[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,12)
ax = plt.gca()
im = ax.imshow(ngal_2deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,13)
ax = plt.gca()
im = ax.imshow(R_selec_4deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,14)
ax = plt.gca()
im = ax.imshow(R_selec_real_std_4deg_from05[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,3,15)
ax = plt.gca()
im = ax.imshow(ngal_4deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)
plt.savefig('./W3/R_selec.png',bbox_inches = 'tight',pad_inches = 0)
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.hist(np.ravel(R_selec_025deg[0,0,:,:]),label=r'$0.25^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(R_selec_05deg[0,0,:,:]),label='1 px = 0.5° x 0.5°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(R_selec_1deg[0,0,:,:]),label=r'$2^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(R_selec_2deg[0,0,:,:]),label='1 px = 2° x 2°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(R_selec_4deg[0,0,:,:]),label=r'$4^\circ$',histtype='step',density=True,bins=50)
plt.axvline(x=R_selec_moy[0,0],c='k',label='global calibration')
plt.legend(fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(-0.2,0)
plt.xlabel(r'$R_{11,selection}$', fontsize=30)
plt.savefig('./W3/Rselec_hist.png',bbox_inches = 'tight',pad_inches = 0.1)
plt.show()

In [ ]:
# Compute % of pixel changed
ind_change_025deg = np.shape((np.where(ngal_025deg < (np.mean(ngal_025deg))/2))[1])
ind_change_05deg = np.shape((np.where(ngal_05deg < (np.mean(ngal_05deg))/2))[1])
ind_change_1deg = np.shape((np.where(ngal_1deg < (np.mean(ngal_1deg))/2))[1])
ind_change_2deg = np.shape((np.where(ngal_2deg < (np.mean(ngal_2deg))/2))[1])
ind_change_4deg = np.shape((np.where(ngal_4deg < (np.mean(ngal_4deg))/2))[1])

In [ ]:
ind_change_01deg = np.shape((np.where(ngal_01deg < (np.mean(ngal_01deg))/2))[1])
ind_change_001deg = np.shape((np.where(ngal_001deg < (np.mean(ngal_001deg))/2))[1])
ind_change_0001deg = np.shape((np.where(ngal_0001deg < (np.mean(ngal_0001deg))/2))[1])
#ind_change_00001deg = np.shape((np.where(ngal_00001deg < (np.mean(ngal_00001deg))/2))[1])

In [ ]:
# Tot size
tot_025deg = (size_deg_x*4)*(size_deg_y*4)
tot_05deg = (size_deg_x*2)*(size_deg_y*2)
tot_1deg = (size_deg_x)*(size_deg_y)
tot_2deg = (int(size_deg_x/2))*(int(size_deg_y/2))
tot_4deg = (int(size_deg_x/4))*(int(size_deg_y/4))

In [ ]:
tot_01deg = (size_deg_x*10)*(size_deg_y*10)
tot_001deg = (size_deg_x*100)*(size_deg_y*100)
tot_0001deg = (size_deg_x*1000)*(size_deg_y*1000)
#tot_00001deg = (size_deg_x*10000)*(size_deg_y*10000)

In [ ]:
# % of mask area
mask_025deg = np.shape(np.where(ngal_025deg==0))[1]
mask_05deg = np.shape(np.where(ngal_05deg==0))[1]
mask_1deg = np.shape(np.where(ngal_1deg==0))[1]
mask_2deg = np.shape(np.where(ngal_2deg==0))[1]
mask_4deg = np.shape(np.where(ngal_4deg==0))[1]

In [ ]:
mask_01deg = np.shape(np.where(ngal_01deg==0))[1]
mask_001deg = np.shape(np.where(ngal_001deg==0))[1]
mask_0001deg = np.shape(np.where(ngal_0001deg==0))[1]
#mask_00001deg = np.shape(np.where(ngal_00001deg==0))[1]

In [ ]:
# % change - % mask
change_prct_025deg = ind_change_025deg[0]*100/tot_025deg - mask_025deg*100/tot_025deg
change_prct_05deg = ind_change_05deg[0]*100/tot_05deg - mask_05deg*100/tot_05deg
change_prct_1deg = ind_change_1deg[0]*100/tot_1deg - mask_1deg*100/tot_1deg
change_prct_2deg = ind_change_2deg[0]*100/tot_2deg - mask_2deg*100/tot_2deg
change_prct_4deg = ind_change_4deg[0]*100/tot_4deg - mask_4deg*100/tot_4deg

In [ ]:
change_prct_01deg = ind_change_01deg[0]*100/tot_01deg - mask_01deg*100/tot_01deg
change_prct_001deg = ind_change_001deg[0]*100/tot_001deg - mask_001deg*100/tot_001deg
change_prct_0001deg = ind_change_0001deg[0]*100/tot_0001deg - mask_0001deg*100/tot_0001deg
#change_prct_00001deg = ind_change_00001deg[0]*100/tot_00001deg - mask_00001deg*100/tot_00001deg

In [ ]:
x=np.array([0.001,0.01,0.1,0.25,0.5,1,2,4])
y=np.array([change_prct_0001deg,change_prct_001deg,change_prct_01deg,change_prct_025deg,change_prct_05deg,change_prct_1deg,change_prct_2deg,change_prct_4deg])

In [ ]:
plt.figure()
plt.scatter(x,y)
plt.xlabel('pixel size in deg')
plt.ylabel('% of value change')
plt.savefig('./P3X/px_change.png')
plt.show()

In [ ]:
# R shear bin needed
R_11, R_22, R_12, R_21 = bin2d(xx, yy, npix=(size_deg_x*4,size_deg_y*4), v=(gal_metacal_ngmix.R_shear[0,0,:],gal_metacal_ngmix.R_shear[1,1,:],gal_metacal_ngmix.R_shear[0,1,:],gal_metacal_ngmix.R_shear[1,0,:]),extent=(min_x,max_x,min_y,max_y)) 
Rmap_025deg = np.array([[R_11,R_12],[R_21,R_22]])

R_11, R_22, R_12, R_21 = bin2d(xx, yy, npix=(size_deg_x*2,size_deg_y*2), v=(gal_metacal_ngmix.R_shear[0,0,:],gal_metacal_ngmix.R_shear[1,1,:],gal_metacal_ngmix.R_shear[0,1,:],gal_metacal_ngmix.R_shear[1,0,:]),extent=(min_x,max_x,min_y,max_y)) 
Rmap_05deg = np.array([[R_11,R_12],[R_21,R_22]])

R_11, R_22, R_12, R_21 = bin2d(xx, yy, npix=(size_deg_x,size_deg_y), v=(gal_metacal_ngmix.R_shear[0,0,:],gal_metacal_ngmix.R_shear[1,1,:],gal_metacal_ngmix.R_shear[0,1,:],gal_metacal_ngmix.R_shear[1,0,:]),extent=(min_x,max_x,min_y,max_y)) 
Rmap_1deg = np.array([[R_11,R_12],[R_21,R_22]])

R_11, R_22, R_12, R_21 = bin2d(xx, yy, npix=(int(size_deg_x/2),int(size_deg_y/2)), v=(gal_metacal_ngmix.R_shear[0,0,:],gal_metacal_ngmix.R_shear[1,1,:],gal_metacal_ngmix.R_shear[0,1,:],gal_metacal_ngmix.R_shear[1,0,:]),extent=(min_x,max_x,min_y,max_y)) 
Rmap_2deg = np.array([[R_11,R_12],[R_21,R_22]])

R_11, R_22, R_12, R_21 = bin2d(xx, yy, npix=(int(size_deg_x/4),int(size_deg_y/4)), v=(gal_metacal_ngmix.R_shear[0,0,:],gal_metacal_ngmix.R_shear[1,1,:],gal_metacal_ngmix.R_shear[0,1,:],gal_metacal_ngmix.R_shear[1,0,:]),extent=(min_x,max_x,min_y,max_y)) 
Rmap_4deg = np.array([[R_11,R_12],[R_21,R_22]])

In [ ]:
Rmap_025deg = np.load('./P3X/var_save/Rmap_025deg.npy')
Rmap_05deg = np.load('./P3X/var_save/Rmap_05deg.npy')
Rmap_1deg = np.load('./P3X/var_save/Rmap_1deg.npy')
Rmap_2deg = np.load('./P3X/var_save/Rmap_2deg.npy')
Rmap_4deg = np.load('./P3X/var_save/Rmap_4deg.npy')

In [ ]:
# Std of R shear
R_shear_real_std_05deg = compute_R_real_std(size_deg_x*2,size_deg_y*2,2,R_shear_025deg)
R_shear_real_std_1deg = compute_R_real_std(size_deg_x,size_deg_y,2,R_shear_05deg)
R_shear_real_std_2deg = compute_R_real_std(int(size_deg_x/2),int(size_deg_y/2),4,R_shear_05deg)
R_shear_real_std_4deg = compute_R_real_std(int(size_deg_x/4),int(size_deg_y/4),8,R_shear_05deg)

In [ ]:
# Plot R shear, R shear bin, ngal
plt.figure(figsize=(70,45))
plt.subplot(5,4,1)
ax = plt.gca()
im = ax.imshow(R_shear_025deg[0,0,:,:],vmin=0.55,vmax=0.9)
plt.title(r'$R_{shear}$',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,2)
ax = plt.gca()
im = ax.imshow(Rmap_025deg[0,0,:,:],vmin=0.55,vmax=0.9)
plt.title(r'$R_{shear}$ binning',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,3)
ax = plt.gca()
plt.title(r'std $R_{shear}$',fontsize=50)
plt.axis('off')

plt.subplot(5,4,4)
ax = plt.gca()
im = ax.imshow(ngal_025deg)
plt.title(r'n gal',fontsize=50)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,5)
ax = plt.gca()
im = ax.imshow(R_shear_05deg[0,0,:,:],vmin=0.66,vmax=0.80)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,6)
ax = plt.gca()
im = ax.imshow(Rmap_05deg[0,0,:,:],vmin=0.66,vmax=0.80)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,7)
ax = plt.gca()
im = ax.imshow(R_shear_real_std_05deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,8)
ax = plt.gca()
im = ax.imshow(ngal_05deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,9)
ax = plt.gca()
im = ax.imshow(R_shear_1deg[0,0,:,:],vmin=0.70,vmax=0.78)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,10)
ax = plt.gca()
im = ax.imshow(Rmap_1deg[0,0,:,:],vmin=0.70,vmax=0.78)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,11)
ax = plt.gca()
im = ax.imshow(R_shear_real_std_1deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,12)
ax = plt.gca()
im = ax.imshow(ngal_1deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,13)
ax = plt.gca()
im = ax.imshow(R_shear_2deg[0,0,:,:],vmin=0.72,vmax=0.76)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,14)
ax = plt.gca()
im = ax.imshow(Rmap_2deg[0,0,:,:],vmin=0.72,vmax=0.76)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)


plt.subplot(5,4,15)
ax = plt.gca()
im = ax.imshow(R_shear_real_std_2deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,16)
ax = plt.gca()
im = ax.imshow(ngal_2deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,17)
ax = plt.gca()
im = ax.imshow(R_shear_4deg[0,0,:,:],vmin=0.71,vmax=0.77)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,18)
ax = plt.gca()
im = ax.imshow(Rmap_4deg[0,0,:,:],vmin=0.71,vmax=0.77)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,19)
ax = plt.gca()
im = ax.imshow(R_shear_real_std_4deg[0,0,:,:])
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)

plt.subplot(5,4,20)
ax = plt.gca()
im = ax.imshow(ngal_4deg)
divider = make_axes_locatable(ax)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im,cax=cax)
plt.yticks(fontsize=35)
plt.savefig('./W3/R_shear.png',bbox_inches = 'tight',pad_inches = 0)
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.hist(np.ravel(R_shear_025deg[0,0,:,:]),label=r'$0.25^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(R_shear_05deg[0,0,:,:]),label='1 px = 0.5° x 0.5°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(R_shear_1deg[0,0,:,:]),label=r'$2^\circ$',histtype='step',density=True,bins=50)
#plt.hist(np.ravel(R_shear_2deg[0,0,:,:]),label='1 px = 2° x 2°',histtype='step',density=True,bins=50)
plt.hist(np.ravel(R_shear_4deg[0,0,:,:]),label=r'$4^\circ$',histtype='step',density=True,bins=50)
plt.axvline(x=R_shear_moy[0,0],c='k',label='global calibration')
plt.legend(fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(0.68,0.81)
plt.xlabel(r'$R_{11,shear}$', fontsize=30)
plt.savefig('./W3/Rshear_hist.png',bbox_inches = 'tight',pad_inches = 0.1)
plt.show()

In [ ]:
ngaltot = bin2d(xx, yy, npix=(Nx,Ny),extent=(min_x,max_x,min_y,max_y))

In [ ]:
ngaltot = np.load('./P3X/var_save/ngaltot.npy')

In [ ]:
def Rtot_extent(npix_x,npix_y,R):
    # From R, create R with same size as g
    
    R_large = np.zeros((Ny, Nx))
    srx = int(Nx / npix_x)
    sry = int(Ny/npix_y)

    for i in range(npix_y):
        for j in range(npix_x):
            r = np.ones((sry, srx)) * R[i,j]
            R_large[i*sry:(i+1)*sry, j*srx:(j+1)*srx] = r
    return R_large

In [ ]:
# Extend R shear from calib 0.5 deg
Rshear_metacal_025deg = np.zeros((2,2,Ny,Nx))
Rshear_metacal_025deg[0,0,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_shear_025deg[0,0,:,:])
Rshear_metacal_025deg[0,1,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_shear_025deg[0,1,:,:])
Rshear_metacal_025deg[1,0,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_shear_025deg[1,0,:,:])
Rshear_metacal_025deg[1,1,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_shear_025deg[1,1,:,:])

# Extend R selec from calib 0.5 deg
Rselec_metacal_025deg = np.zeros((2,2,Ny,Nx))
Rselec_metacal_025deg[0,0,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_selec_025deg[0,0,:,:])
Rselec_metacal_025deg[0,1,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_selec_025deg[0,1,:,:])
Rselec_metacal_025deg[1,0,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_selec_025deg[1,0,:,:])
Rselec_metacal_025deg[1,1,:,:] = Rtot_extent(size_deg_x*4,size_deg_y*4,R_selec_025deg[1,1,:,:])

In [ ]:
# Extend c from calib 0.5 deg
c1_ngmix_metacal_025deg = Rtot_extent(size_deg_x*4,size_deg_y*4,c1_ngmix_025deg)
c2_ngmix_metacal_025deg = Rtot_extent(size_deg_x*4,size_deg_y*4,c2_ngmix_025deg)

In [ ]:
# Extend R shear from calib 0.5 deg
Rshear_metacal_05deg = np.zeros((2,2,Ny,Nx))
Rshear_metacal_05deg[0,0,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_shear_05deg[0,0,:,:])
Rshear_metacal_05deg[0,1,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_shear_05deg[0,1,:,:])
Rshear_metacal_05deg[1,0,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_shear_05deg[1,0,:,:])
Rshear_metacal_05deg[1,1,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_shear_05deg[1,1,:,:])

# Extend R selec from calib 0.5 deg
Rselec_metacal_05deg = np.zeros((2,2,Ny,Nx))
Rselec_metacal_05deg[0,0,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_selec_05deg[0,0,:,:])
Rselec_metacal_05deg[0,1,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_selec_05deg[0,1,:,:])
Rselec_metacal_05deg[1,0,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_selec_05deg[1,0,:,:])
Rselec_metacal_05deg[1,1,:,:] = Rtot_extent(size_deg_x*2,size_deg_y*2,R_selec_05deg[1,1,:,:])

In [ ]:
# Extend c from calib 0.5 deg
c1_ngmix_metacal_05deg = Rtot_extent(size_deg_x*2,size_deg_y*2,c1_ngmix_05deg)
c2_ngmix_metacal_05deg = Rtot_extent(size_deg_x*2,size_deg_y*2,c2_ngmix_05deg)

In [ ]:
# Extend R shear from calib 1 deg
Rshear_metacal_1deg = np.zeros((2,2,Ny,Nx))
Rshear_metacal_1deg[0,0,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_shear_1deg[0,0,:,:])
Rshear_metacal_1deg[0,1,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_shear_1deg[0,1,:,:])
Rshear_metacal_1deg[1,0,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_shear_1deg[1,0,:,:])
Rshear_metacal_1deg[1,1,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_shear_1deg[1,1,:,:])

# Extend R selec from calib 1 deg
Rselec_metacal_1deg = np.zeros((2,2,Ny,Nx))
Rselec_metacal_1deg[0,0,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_selec_1deg[0,0,:,:])
Rselec_metacal_1deg[0,1,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_selec_1deg[0,1,:,:])
Rselec_metacal_1deg[1,0,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_selec_1deg[1,0,:,:])
Rselec_metacal_1deg[1,1,:,:] = Rtot_extent(size_deg_x,size_deg_y,R_selec_1deg[1,1,:,:])

In [ ]:
# Extend c from calib 1 deg
c1_ngmix_metacal_1deg = Rtot_extent(size_deg_x,size_deg_y,c1_ngmix_1deg)
c2_ngmix_metacal_1deg = Rtot_extent(size_deg_x,size_deg_y,c2_ngmix_1deg)

In [ ]:
# Extend R shear from calib 2 deg
Rshear_metacal_2deg = np.zeros((2,2,Ny,Nx))
Rshear_metacal_2deg[0,0,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_shear_2deg[0,0,:,:])
Rshear_metacal_2deg[0,1,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_shear_2deg[0,1,:,:])
Rshear_metacal_2deg[1,0,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_shear_2deg[1,0,:,:])
Rshear_metacal_2deg[1,1,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_shear_2deg[1,1,:,:])

# Extend R selec from calib 2 deg
Rselec_metacal_2deg = np.zeros((2,2,Ny,Nx))
Rselec_metacal_2deg[0,0,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_selec_2deg[0,0,:,:])
Rselec_metacal_2deg[0,1,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_selec_2deg[0,1,:,:])
Rselec_metacal_2deg[1,0,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_selec_2deg[1,0,:,:])
Rselec_metacal_2deg[1,1,:,:] = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),R_selec_2deg[1,1,:,:])

In [ ]:
# Extend c from calib 2 deg
c1_ngmix_metacal_2deg = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),c1_ngmix_2deg)
c2_ngmix_metacal_2deg = Rtot_extent(int(size_deg_x/2),int(size_deg_y/2),c2_ngmix_2deg)

In [ ]:
# Extend R shear from calib 2 deg
Rshear_metacal_4deg = np.zeros((2,2,Ny,Nx))
Rshear_metacal_4deg[0,0,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_shear_4deg[0,0,:,:])
Rshear_metacal_4deg[0,1,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_shear_4deg[0,1,:,:])
Rshear_metacal_4deg[1,0,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_shear_4deg[1,0,:,:])
Rshear_metacal_4deg[1,1,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_shear_4deg[1,1,:,:])

# Extend R selec from calib 2 deg
Rselec_metacal_4deg = np.zeros((2,2,Ny,Nx))
Rselec_metacal_4deg[0,0,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_selec_4deg[0,0,:,:])
Rselec_metacal_4deg[0,1,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_selec_4deg[0,1,:,:])
Rselec_metacal_4deg[1,0,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_selec_4deg[1,0,:,:])
Rselec_metacal_4deg[1,1,:,:] = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),R_selec_4deg[1,1,:,:])

In [ ]:
# Extend c from calib 4 deg
c1_ngmix_metacal_4deg = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),c1_ngmix_4deg)
c2_ngmix_metacal_4deg = Rtot_extent(int(size_deg_x/4),int(size_deg_y/4),c2_ngmix_4deg)

In [ ]:
# Check
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.imshow(R_shear_05deg[0,0,:,:])
plt.colorbar()
plt.clim(0.7,0.77)
plt.subplot(1,2,2)
plt.imshow(Rshear_metacal_05deg[0,0,:,:])
plt.colorbar()
plt.clim(0.7,0.77)
plt.show()
print(np.where(Rshear_metacal_05deg[0,0,:,:]==0))

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.imshow(R_selec_05deg[0,0,:,:])
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(Rselec_metacal_05deg[0,0,:,:])
plt.colorbar()
plt.show()
print(np.where(Rselec_metacal_05deg[0,0,:,:]==0))

In [ ]:
def bin_Rtot(Rshear,Rselec):
    # R tot = R shear + R selection
    
    Rtot = np.zeros((2,2,Ny,Nx))
    for i in range(Ny):
        for j in range(Nx):
            Rtot[:,:,i,j] = Rshear[:,:,i,j] + Rselec[:,:,i,j]
    return Rtot

In [ ]:
Rtot_metacal_05deg = bin_Rtot(Rshear_metacal_05deg,Rselec_metacal_05deg)
Rtot_metacal_1deg = bin_Rtot(Rshear_metacal_1deg,Rselec_metacal_1deg)
Rtot_metacal_2deg = bin_Rtot(Rshear_metacal_2deg,Rselec_metacal_2deg)
Rtot_metacal_4deg = bin_Rtot(Rshear_metacal_4deg,Rselec_metacal_4deg)

Kappa map

In [ ]:
# Kappa map
# Global calibration
kappaE_cal, kappaB_cal = ks93(emap[0]-c1_ngmix,-(emap[1]-c2_ngmix))

# From local calibration
kappaE_05deg, kappaB_05deg = ks93(gmap_05deg[0]-c1_ngmix_metacal_05deg, -(gmap_05deg[1]-c2_ngmix_metacal_05deg)) 
kappaE_1deg, kappaB_1deg = ks93(gmap_1deg[0]-c1_ngmix_metacal_1deg, -(gmap_1deg[1]-c2_ngmix_metacal_1deg)) 
kappaE_2deg, kappaB_2deg = ks93(gmap_2deg[0]-c1_ngmix_metacal_2deg, -(gmap_2deg[1]-c2_ngmix_metacal_2deg)) 
kappaE_4deg, kappaB_4deg = ks93(gmap_4deg[0]-c1_ngmix_metacal_4deg, -(gmap_4deg[1]-c2_ngmix_metacal_4deg))

#kappaE_05deg_globalc, kappaB_05deg_globalc = ks93(gmap_05deg[0]-c1_ngmix, -(gmap_05deg[1]-c2_ngmix)) 
#kappaE_1deg_globalc, kappaB_1deg_globalc = ks93(gmap_1deg[0]-c1_ngmix, -(gmap_1deg[1]-c2_ngmix)) 
#kappaE_2deg_globalc, kappaB_2deg_globalc = ks93(gmap_2deg[0]-c1_ngmix, -(gmap_2deg[1]-c2_ngmix)) 
#kappaE_4deg_globalc, kappaB_4deg_globalc = ks93(gmap_4deg[0]-c1_ngmix, -(gmap_4deg[1]-c2_ngmix))

# From local calibration + dm
#kappaE_dm_05deg, kappaB_dm_05deg = ks93(gmap_dm_05deg[0]-c1_ngmix_metacal_05deg, -(gmap_dm_05deg[1]-c2_ngmix_metacal_05deg))
#kappaE_dm_1deg, kappaB_dm_1deg = ks93(gmap_dm_1deg[0]-c1_ngmix_metacal_1deg, -(gmap_dm_1deg[1]-c2_ngmix_metacal_1deg))
#kappaE_dm_2deg, kappaB_dm_2deg = ks93(gmap_dm_2deg[0]-c1_ngmix_metacal_2deg, -(gmap_dm_2deg[1]-c2_ngmix_metacal_2deg)) 
#kappaE_dm_4deg, kappaB_dm_4deg = ks93(gmap_dm_4deg[0]-c1_ngmix_metacal_4deg, -(gmap_dm_4deg[1]-c2_ngmix_metacal_4deg))

In [ ]:
# From local calibration + dm = -0.0208
kappaE_dm_05deg_ax, kappaB_dm_05deg_ax = ks93(gmap_dm_05deg_ax[0]-c1_ngmix_metacal_05deg, -(gmap_dm_05deg_ax[1]-c2_ngmix_metacal_05deg))
kappaE_dm_1deg_ax, kappaB_dm_1deg_ax = ks93(gmap_dm_1deg_ax[0]-c1_ngmix_metacal_1deg, -(gmap_dm_1deg_ax[1]-c2_ngmix_metacal_1deg))
kappaE_dm_2deg_ax, kappaB_dm_2deg_ax = ks93(gmap_dm_2deg_ax[0]-c1_ngmix_metacal_2deg, -(gmap_dm_2deg_ax[1]-c2_ngmix_metacal_2deg)) 
kappaE_dm_4deg_ax, kappaB_dm_4deg_ax = ks93(gmap_dm_4deg_ax[0]-c1_ngmix_metacal_4deg, -(gmap_dm_4deg_ax[1]-c2_ngmix_metacal_4deg))

In [ ]:
kappaE_05deg = fits.getdata('./P3X/kappa_map/kappaE_05deg.fits')
kappaE_1deg = fits.getdata('./P3X/kappa_map/kappaE_1deg.fits')
kappaE_2deg = fits.getdata('./P3X/kappa_map/kappaE_2deg.fits')
kappaE_4deg = fits.getdata('./P3X/kappa_map/kappaE_4deg.fits')

In [ ]:
def reduce(npix_x,npix_y,kappa):
    # From kappa, create kappa_small to mimic the binning
    
    kappa_small = np.zeros((npix_y, npix_x))
    srx = int(Nx/npix_x)
    sry = int(Ny/npix_y)

    for i in range(npix_y):
        for j in range(npix_x):
            a = np.mean(kappa[i*sry:(i+1)*sry, j*srx:(j+1)*srx])
            kappa_small[i,j] = a
    return kappa_small

In [ ]:
# Reduce map usefull for the random forest

In [ ]:
gmap_05deg_comp1 = reduce(72,40,gmap_05deg[0])
gmap_05deg_comp2 = reduce(72,40,gmap_05deg[1])

In [ ]:
kappa_small_05deg = reduce(72,40,kappaE_05deg)
kappa_small_1deg = reduce(36,20,kappaE_1deg)
kappa_small_2deg = reduce(18,10,kappaE_2deg)
kappa_small_4deg = reduce(9,5,kappaE_4deg)

In [ ]:
plt.figure(figsize=(20,9))
plt.subplot(1,2,1)
plt.imshow(gmap_05deg_comp1)
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(gmap_05deg[0])
plt.colorbar()
plt.show()

In [ ]:
hdu = fits.PrimaryHDU(kappaE_dm_4deg_ax)
hdul = fits.HDUList([hdu])
hdul.writeto('./W3/kappaE_dm_4deg_ax.fits')

In [ ]:
#galaxy number density in gal/arcmin^2
n_gal=7 # gal/arcmin^2 in the survey

#pixel size in arcmin
pix_arcmin=0.4

#compute shape noise for CFIS
shape_noise=0.44
sigma_noise_CFIS=shape_noise/(np.sqrt(2*n_gal*pix_arcmin**2))
noise_CFIS_z05=sigma_noise_CFIS*np.random.randn(Ny, Nx)

In [ ]:
#smooth the map with Gaussian kernel
precision_Peaks = 2/pix_arcmin

kappa_noisy_smoothedB = ndi.gaussian_filter(kappaB_cal, precision_Peaks) 
kappa_noisy_smoothed_cal = ndi.gaussian_filter(kappaE_cal, precision_Peaks)

kappa_noisy_smoothed_05deg = ndi.gaussian_filter(kappaE_05deg, precision_Peaks)
kappa_noisy_smoothed_1deg = ndi.gaussian_filter(kappaE_1deg, precision_Peaks)
kappa_noisy_smoothed_2deg = ndi.gaussian_filter(kappaE_2deg, precision_Peaks)
kappa_noisy_smoothed_4deg = ndi.gaussian_filter(kappaE_4deg, precision_Peaks)

#kappa_noisy_smoothed_dm_1deg = ndi.gaussian_filter(kappaE_dm_1deg, precision_Peaks)
#kappa_noisy_smoothed_dm_2deg = ndi.gaussian_filter(kappaE_dm_2deg, precision_Peaks)
#kappa_noisy_smoothed_dm_4deg = ndi.gaussian_filter(kappaE_dm_4deg, precision_Peaks)

In [ ]:
plt.figure(figsize=(25,10))
plt.subplot(1,2,1)
ax = plt.gca()
im = ax.imshow(kappa_noisy_smoothed_cal)
divider = make_axes_locatable(ax)
plt.title('Global')
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

plt.subplot(1,2,2)
ax = plt.gca()
im = ax.imshow(kappa_noisy_smoothed_1deg)
divider = make_axes_locatable(ax)
plt.title('metacal deg')
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.show()

In [ ]:
# snr map
snr_cal = kappa_noisy_smoothed_cal/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))
snrB = kappa_noisy_smoothedB/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))

snr_05deg = kappa_noisy_smoothed_05deg/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))
snr_1deg = kappa_noisy_smoothed_1deg/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))
snr_2deg = kappa_noisy_smoothed_2deg/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))
snr_4deg = kappa_noisy_smoothed_4deg/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))

In [ ]:
# Compute peak count on kappa smooth
kappa_th = np.linspace(-0.06, 0.06, 31)
kappa_th_center = 0.5*(kappa_th[:-1]+kappa_th[1:])

peak_counts_cal, bins_cal = peaks.peaks_histogram(kappa_noisy_smoothed_cal,kappa_th)

peak_counts_1deg, bins = peaks.peaks_histogram(kappa_noisy_smoothed_1deg,kappa_th)
peak_counts_2deg, bins = peaks.peaks_histogram(kappa_noisy_smoothed_2deg,kappa_th)
peak_counts_4deg, bins = peaks.peaks_histogram(kappa_noisy_smoothed_4deg,kappa_th)

In [ ]:
# Compute peaks on snr map
kappa_snr = np.linspace(-2,6,31)
kappa_th_center_snr = 0.5*(kappa_snr[:-1]+kappa_snr[1:])

peak_counts_cal_snr, bins_cal = peaks.peaks_histogram(snr_cal,kappa_snr)

peak_counts_1deg_snr, bins = peaks.peaks_histogram(snr_1deg,kappa_snr)
peak_counts_2deg_snr, bins = peaks.peaks_histogram(snr_2deg,kappa_snr)
peak_counts_4deg_snr, bins = peaks.peaks_histogram(snr_4deg,kappa_snr)

In [ ]:
#plot single-Gaussian peak counts : Verification
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
plt.plot(kappa_th_center,peak_counts_cal,label='Global cal')
plt.legend()
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('kappa smooth', fontsize=18)
plt.ylabel('peak counts', fontsize=18)
plt.subplot(1,2,2)
plt.plot(kappa_th_center_snr,peak_counts_cal_snr)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('snr', fontsize=18)
plt.ylabel('peak counts', fontsize=18)
plt.show()

In [ ]:
# For random forest
snr_small_05deg = reduce(72,40,snr_05deg)
snr_small_1deg = reduce(36,20,snr_1deg)
snr_small_2deg = reduce(18,10,snr_2deg)
snr_small_4deg = reduce(9,5,snr_4deg)

In [ ]:
# Find the peaks positions and bin it
y_peaks, x_peaks, h_peaks = find_peaks2d(snr_cal, threshold=-2, include_border=True)
pos_peaks = bin2d(x_peaks, y_peaks, npix=(Nx,Ny))

plt.figure()
plt.imshow(pos_peaks)
plt.colorbar()
plt.clim(-0.01,0.01)
plt.show()

In [ ]:
# Pos peak for random forest
y_peaks, x_peaks, h_peaks = find_peaks2d(snr_05deg, threshold=-2, include_border=True)
pos_peaks_05deg = bin2d(x_peaks, y_peaks, npix=(Nx,Ny))

y_peaks, x_peaks, h_peaks = find_peaks2d(snr_1deg, threshold=-2, include_border=True)
pos_peaks_1deg = bin2d(x_peaks, y_peaks, npix=(Nx,Ny))

y_peaks, x_peaks, h_peaks = find_peaks2d(snr_2deg, threshold=-2, include_border=True)
pos_peaks_2deg = bin2d(x_peaks, y_peaks, npix=(Nx,Ny))

y_peaks, x_peaks, h_peaks = find_peaks2d(snr_4deg, threshold=-2, include_border=True)
pos_peaks_4deg = bin2d(x_peaks, y_peaks, npix=(Nx,Ny))

In [ ]:
plt.figure()
plt.imshow(pos_peaks_4deg)
plt.colorbar()
plt.clim(-0.01,0.01)
plt.show()

In [ ]:
# Reduce pos peaks for random forest
pos_peaks_small_05deg = reduce(72,40,pos_peaks_05deg)
pos_peaks_small_1deg = reduce(36,20,pos_peaks_1deg)
pos_peaks_small_2deg = reduce(18,10,pos_peaks_2deg)
pos_peaks_small_4deg = reduce(9,5,pos_peaks_4deg)

In [ ]:
plt.figure()
plt.imshow(pos_peaks_small_4deg)
plt.colorbar()
plt.clim(-0.01,0.01)
plt.show()

In [ ]:
pos_peaks = np.load('./P3X/var_save/pos_peaks.npy')

Cross correlation

In [ ]:
# R shear, selec, tot diagonal and off-diag
Rshear_diag_1deg = (Rshear_metacal_1deg[0,0,:,:]+Rshear_metacal_1deg[1,1,:,:])/2
Rshear_off_diag_1deg = (Rshear_metacal_1deg[0,1,:,:]+Rshear_metacal_1deg[1,0,:,:])/2

Rselec_diag_1deg = (Rselec_metacal_1deg[0,0,:,:]+Rselec_metacal_1deg[1,1,:,:])/2
Rselec_off_diag_1deg = (Rselec_metacal_1deg[0,1,:,:]+Rselec_metacal_1deg[1,0,:,:])/2

Rtot_diag_1deg = (Rtot_metacal_1deg[0,0,:,:]+Rtot_metacal_1deg[1,1,:,:])/2
Rtot_off_diag_1deg = (Rtot_metacal_1deg[0,1,:,:]+Rtot_metacal_1deg[1,0,:,:])/2

In [ ]:
# R shear, selec, tot diagonal and off-diag
Rshear_diag_05deg = (Rshear_metacal_05deg[0,0,:,:]+Rshear_metacal_05deg[1,1,:,:])/2
Rshear_off_diag_05deg = (Rshear_metacal_05deg[0,1,:,:]+Rshear_metacal_05deg[1,0,:,:])/2

Rselec_diag_05deg = (Rselec_metacal_05deg[0,0,:,:]+Rselec_metacal_05deg[1,1,:,:])/2
Rselec_off_diag_05deg = (Rselec_metacal_05deg[0,1,:,:]+Rselec_metacal_05deg[1,0,:,:])/2

Rtot_diag_05deg = (Rtot_metacal_05deg[0,0,:,:]+Rtot_metacal_05deg[1,1,:,:])/2
Rtot_off_diag_05deg = (Rtot_metacal_05deg[0,1,:,:]+Rtot_metacal_05deg[1,0,:,:])/2

In [ ]:
Rshear_diag_2deg = (Rshear_metacal_2deg[0,0,:,:]+Rshear_metacal_2deg[1,1,:,:])/2
Rshear_off_diag_2deg = (Rshear_metacal_2deg[0,1,:,:]+Rshear_metacal_2deg[1,0,:,:])/2

Rselec_diag_2deg = (Rselec_metacal_2deg[0,0,:,:]+Rselec_metacal_2deg[1,1,:,:])/2
Rselec_off_diag_2deg = (Rselec_metacal_2deg[0,1,:,:]+Rselec_metacal_2deg[1,0,:,:])/2

Rtot_diag_2deg = (Rtot_metacal_2deg[0,0,:,:]+Rtot_metacal_2deg[1,1,:,:])/2
Rtot_off_diag_2deg = (Rtot_metacal_2deg[0,1,:,:]+Rtot_metacal_2deg[1,0,:,:])/2

Rshear_diag_4deg = (Rshear_metacal_4deg[0,0,:,:]+Rshear_metacal_4deg[1,1,:,:])/2
Rshear_off_diag_4deg = (Rshear_metacal_4deg[0,1,:,:]+Rshear_metacal_4deg[1,0,:,:])/2

Rselec_diag_4deg = (Rselec_metacal_4deg[0,0,:,:]+Rselec_metacal_4deg[1,1,:,:])/2
Rselec_off_diag_4deg = (Rselec_metacal_4deg[0,1,:,:]+Rselec_metacal_4deg[1,0,:,:])/2

Rtot_diag_4deg = (Rtot_metacal_4deg[0,0,:,:]+Rtot_metacal_4deg[1,1,:,:])/2
Rtot_off_diag_4deg = (Rtot_metacal_4deg[0,1,:,:]+Rtot_metacal_4deg[1,0,:,:])/2

In [ ]:
# Find area where low number of galaxy
a = np.where(ngaltot > (np.mean(ngaltot))/2)

In [ ]:
data = {'n gal': np.ravel(ngaltot[a]),
        'kappa_E': np.ravel(kappaE_cal[a]),
        'kappa_E smooth': np.ravel(kappa_noisy_smoothed_cal[a]),
        'kappa_B': np.ravel(kappaB_cal[a]),
        'kappa_B smooth': np.ravel(kappa_noisy_smoothedB[a]),
        'kappa_E 05 deg': np.ravel(kappaE_05deg[a]),
        'kappa_B 05 deg': np.ravel(kappaB_05deg[a]),
        'kappa_E 1 deg': np.ravel(kappaE_1deg[a]),
        'kappa_B 1 deg': np.ravel(kappaB_1deg[a]),
        'kappa_E 2 deg': np.ravel(kappaE_2deg[a]),
        'kappa_B 2 deg': np.ravel(kappaB_2deg[a]),
        'kappa_E 4 deg': np.ravel(kappaE_4deg[a]),
        'kappa_B 4 deg': np.ravel(kappaB_4deg[a]),
        'R_11 shear 05 deg': np.ravel(Rshear_metacal_05deg[0,0,:,:][a]),
        'R_22 shear 05 deg': np.ravel(Rshear_metacal_05deg[1,1,:,:][a]),
        'R_12 shear 05 deg': np.ravel(Rshear_metacal_05deg[0,1,:,:][a]),
        'R_21 shear 05 deg': np.ravel(Rshear_metacal_05deg[1,0,:,:][a]),
        'R_11 selec 05 deg': np.ravel(Rselec_metacal_05deg[0,0,:,:][a]),
        'R_22 selec 05 deg': np.ravel(Rselec_metacal_05deg[1,1,:,:][a]),
        'R_12 selec 05 deg': np.ravel(Rselec_metacal_05deg[0,1,:,:][a]),
        'R_21 selec 05 deg': np.ravel(Rselec_metacal_05deg[1,0,:,:][a]),
        'R_11 tot 05 deg': np.ravel(Rtot_metacal_05deg[0,0,:,:][a]),
        'R_22 tot 05 deg': np.ravel(Rtot_metacal_05deg[1,1,:,:][a]),
        'R_12 tot 05 deg': np.ravel(Rtot_metacal_05deg[0,1,:,:][a]),
        'R_21 tot 05 deg': np.ravel(Rtot_metacal_05deg[1,0,:,:][a]),
        'R_11 shear 1 deg': np.ravel(Rshear_metacal_1deg[0,0,:,:][a]),
        'R_22 shear 1 deg': np.ravel(Rshear_metacal_1deg[1,1,:,:][a]),
        'R_12 shear 1 deg': np.ravel(Rshear_metacal_1deg[0,1,:,:][a]),
        'R_21 shear 1 deg': np.ravel(Rshear_metacal_1deg[1,0,:,:][a]),
        'R_11 selec 1 deg': np.ravel(Rselec_metacal_1deg[0,0,:,:][a]),
        'R_22 selec 1 deg': np.ravel(Rselec_metacal_1deg[1,1,:,:][a]),
        'R_12 selec 1 deg': np.ravel(Rselec_metacal_1deg[0,1,:,:][a]),
        'R_21 selec 1 deg': np.ravel(Rselec_metacal_1deg[1,0,:,:][a]),
        'R_11 tot 1 deg': np.ravel(Rtot_metacal_1deg[0,0,:,:][a]),
        'R_22 tot 1 deg': np.ravel(Rtot_metacal_1deg[1,1,:,:][a]),
        'R_12 tot 1 deg': np.ravel(Rtot_metacal_1deg[0,1,:,:][a]),
        'R_21 tot 1 deg': np.ravel(Rtot_metacal_1deg[1,0,:,:][a]),
        'R_11 shear 2 deg': np.ravel(Rshear_metacal_2deg[0,0,:,:][a]),
        'R_22 shear 2 deg': np.ravel(Rshear_metacal_2deg[1,1,:,:][a]),
        'R_12 shear 2 deg': np.ravel(Rshear_metacal_2deg[0,1,:,:][a]),
        'R_21 shear 2 deg': np.ravel(Rshear_metacal_2deg[1,0,:,:][a]),
        'R_11 selec 2 deg': np.ravel(Rselec_metacal_2deg[0,0,:,:][a]),
        'R_22 selec 2 deg': np.ravel(Rselec_metacal_2deg[1,1,:,:][a]),
        'R_12 selec 2 deg': np.ravel(Rselec_metacal_2deg[0,1,:,:][a]),
        'R_21 selec 2 deg': np.ravel(Rselec_metacal_2deg[1,0,:,:][a]),
        'R_11 tot 2 deg': np.ravel(Rtot_metacal_2deg[0,0,:,:][a]),
        'R_22 tot 2 deg': np.ravel(Rtot_metacal_2deg[1,1,:,:][a]),
        'R_12 tot 2 deg': np.ravel(Rtot_metacal_2deg[0,1,:,:][a]),
        'R_21 tot 2 deg': np.ravel(Rtot_metacal_2deg[1,0,:,:][a]),
        'R_11 shear 4 deg': np.ravel(Rshear_metacal_4deg[0,0,:,:][a]),
        'R_22 shear 4 deg': np.ravel(Rshear_metacal_4deg[1,1,:,:][a]),
        'R_12 shear 4 deg': np.ravel(Rshear_metacal_4deg[0,1,:,:][a]),
        'R_21 shear 4 deg': np.ravel(Rshear_metacal_4deg[1,0,:,:][a]),
        'R_11 selec 4 deg': np.ravel(Rselec_metacal_4deg[0,0,:,:][a]),
        'R_22 selec 4 deg': np.ravel(Rselec_metacal_4deg[1,1,:,:][a]),
        'R_12 selec 4 deg': np.ravel(Rselec_metacal_4deg[0,1,:,:][a]),
        'R_21 selec 4 deg': np.ravel(Rselec_metacal_4deg[1,0,:,:][a]),
        'R_11 tot 4 deg': np.ravel(Rtot_metacal_4deg[0,0,:,:][a]),
        'R_22 tot 4 deg': np.ravel(Rtot_metacal_4deg[1,1,:,:][a]),
        'R_12 tot 4 deg': np.ravel(Rtot_metacal_4deg[0,1,:,:][a]),
        'R_21 tot 4 deg': np.ravel(Rtot_metacal_4deg[1,0,:,:][a]),
        'psf 1': np.ravel(psf1map[a]),
        'psf 2': np.ravel(psf2map[a]),
        'module psf': np.ravel(module_psf_map[a]),
        'size gal': np.ravel(size_gal_map[a]),
        'n epoch': np.ravel(nepoch_map[a]),
        'mag': np.ravel(mag_map[a]),
        'snr': np.ravel(snr_map[a]),
        'snr ngmix': np.ravel(snr_ngmix_map[a]),
        'size psf': np.ravel(tpsf_map[a]),
        'weight gal': np.ravel(w_ngmix_map[a]),
        'R diag shear 05 deg': np.ravel(Rshear_diag_05deg[a]),
        'R off diag shear 05 deg': np.ravel(Rshear_off_diag_05deg[a]),
        'R diag selec 05 deg': np.ravel(Rselec_diag_05deg[a]),
        'R off diag selec 05 deg': np.ravel(Rselec_off_diag_05deg[a]),
        'R diag tot 05 deg': np.ravel(Rtot_diag_05deg[a]),
        'R off diag tot 05 deg': np.ravel(Rtot_off_diag_05deg[a]),
        'R diag shear 1 deg': np.ravel(Rshear_diag_1deg[a]),
        'R off diag shear 1 deg': np.ravel(Rshear_off_diag_1deg[a]),
        'R diag selec 1 deg': np.ravel(Rselec_diag_1deg[a]),
        'R off diag selec 1 deg': np.ravel(Rselec_off_diag_1deg[a]),
        'R diag tot 1 deg': np.ravel(Rtot_diag_1deg[a]),
        'R off diag tot 1 deg': np.ravel(Rtot_off_diag_1deg[a]),
        'R diag shear 2 deg': np.ravel(Rshear_diag_2deg[a]),
        'R off diag shear 2 deg': np.ravel(Rshear_off_diag_2deg[a]),
        'R diag selec 2 deg': np.ravel(Rselec_diag_2deg[a]),
        'R off diag selec 2 deg': np.ravel(Rselec_off_diag_2deg[a]),
        'R diag tot 2 deg': np.ravel(Rtot_diag_2deg[a]),
        'R off diag tot 2 deg': np.ravel(Rtot_off_diag_2deg[a]),
        'R diag shear 4 deg': np.ravel(Rshear_diag_4deg[a]),
        'R off diag shear 4 deg': np.ravel(Rshear_off_diag_4deg[a]),
        'R diag selec 4 deg': np.ravel(Rselec_diag_4deg[a]),
        'R off diag selec 4 deg': np.ravel(Rselec_off_diag_4deg[a]),
        'R diag tot 4 deg': np.ravel(Rtot_diag_4deg[a]),
        'R off diag tot 4 deg': np.ravel(Rtot_off_diag_4deg[a]),
        'peaks': np.ravel(pos_peaks[a]),
        'c1 05 deg': np.ravel(c1_ngmix_metacal_05deg[a]),
        'c2 05 deg': np.ravel(c2_ngmix_metacal_05deg[a]),
        'c1 1 deg': np.ravel(c1_ngmix_metacal_1deg[a]),
        'c2 1 deg': np.ravel(c2_ngmix_metacal_1deg[a]),
        'c1 2 deg': np.ravel(c1_ngmix_metacal_2deg[a]),
        'c2 2 deg': np.ravel(c2_ngmix_metacal_2deg[a]),
        'c1 4 deg': np.ravel(c1_ngmix_metacal_4deg[a]),
        'c2 4 deg': np.ravel(c2_ngmix_metacal_4deg[a])
        }

In [ ]:
df_all = pd.DataFrame(data)
df_all.to_csv('./P3X/all_data.csv',index=False)

In [ ]:
# For random forest
#Rsh_diag_05deg = (R_shear_05deg[0,0,:,:]+R_shear_05deg[1,1,:,:])/2
#Rse_diag_05deg = (R_selec_05deg[0,0,:,:]+R_selec_05deg[1,1,:,:])/2

#Rsh_diag_1deg = (R_shear_1deg[0,0,:,:]+R_shear_1deg[1,1,:,:])/2
#Rse_diag_1deg = (R_selec_1deg[0,0,:,:]+R_selec_1deg[1,1,:,:])/2

#Rsh_diag_2deg = (R_shear_2deg[0,0,:,:]+R_shear_2deg[1,1,:,:])/2
#Rse_diag_2deg = (R_selec_2deg[0,0,:,:]+R_selec_2deg[1,1,:,:])/2

#Rsh_diag_4deg = (R_shear_4deg[0,0,:,:]+R_shear_4deg[1,1,:,:])/2
#Rse_diag_4deg = (R_selec_4deg[0,0,:,:]+R_selec_4deg[1,1,:,:])/2

#data_rf = {'n gal 05 deg': np.ravel(ngal_05deg),
#        'gmap1 05 deg' : np.ravel(gmap_05deg_comp1),
#        'gmap2 05 deg' : np.ravel(gmap_05deg_comp2),
#        'snr 05 deg': np.ravel(snr_small_05deg),
#        'module psf 05 deg': np.ravel(module_psf_map_05deg),
#        'size gal 05 deg': np.ravel(size_gal_map_05deg),
#        'n epoch 05 deg': np.ravel(nepoch_map_05deg),
#        'mag 05 deg': np.ravel(mag_map_05deg),
#        'snr 05 deg': np.ravel(snr_map_05deg),
#        'c1 05 deg': np.ravel(c1_ngmix_05deg),
#        'c2 05 deg': np.ravel(c2_ngmix_05deg),
#        'pos peak 05 deg': np.ravel(pos_peaks_small_05deg),
#        'R diag shear 05 deg': np.ravel(Rsh_diag_05deg),
#        'R diag selec 05 deg': np.ravel(Rse_diag_05deg)
#        }

#df_rf = pd.DataFrame(data_rf)
#df_rf.to_csv('./P3X/random_forest/data_for_random_forest_05deg_new.csv',index=False)

In [ ]:
cmap = sns.diverging_palette(250, 20, as_cmap=True,n=200)

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [ ]:
new_cmap=discrete_cmap(10,'RdBu')

In [ ]:
df = pd.DataFrame(data,columns=['n gal','kappa_E','kappa_E smooth','kappa_B','kappa_B smooth','kappa_E 1 deg',
                                'kappa_B 1 deg','R_11 shear 1 deg','R_22 shear 1 deg','R_12 shear 1 deg',
                                'R_21 shear 1 deg','R_11 selec 1 deg','R_22 selec 1 deg', 'R_12 selec 1 deg',
                                'R_21 selec 1 deg','R_11 tot 1 deg','R_22 tot 1 deg','R_12 tot 1 deg',
                                'R_21 tot 1 deg','psf 1','psf 2','module psf','size gal','n epoch','mag','size psf',
                                'snr','weight gal','peaks','c1 1 deg','c2 1 deg'])

df2 = pd.DataFrame(data,columns=['n gal','kappa_E 1 deg','kappa_B 1 deg','R diag shear 1 deg','R off diag shear 1 deg',
                                 'R diag selec 1 deg','R off diag selec 1 deg','R diag tot 1 deg',
                                 'R off diag tot 1 deg','module psf','c1 1 deg','c2 1 deg'])

In [ ]:
corrMatrix = df.corr()
plt.figure(figsize=(15,15))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=False, annot_kws={"fontsize":8},vmin=-1,vmax=1.,linewidth=0.2)
plt.savefig('./P3X/matrix_corr.png')
plt.show()

In [ ]:
from scipy import stats

In [ ]:
stats.pearsonr(psf1map[a],c1_ngmix_metacal_1deg[a])

In [ ]:
corrMatrix = df2.corr()
plt.figure(figsize=(12,12))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=True, annot_kws={"fontsize":9},vmin=-1,vmax=1.)
plt.savefig('./P3X/matrix_corr_reduite.png',bbox_inches="tight")
plt.show()

In [ ]:
df_2deg = pd.DataFrame(data,columns=['n gal','kappa_E','kappa_E smooth','kappa_B','kappa_B smooth','kappa_E 2 deg',
                                'kappa_B 2 deg','R_11 shear 2 deg','R_22 shear 2 deg','R_12 shear 2 deg','R_21 shear 2 deg',
                                'R_11 selec 2 deg','R_22 selec 2 deg','R_12 selec 2 deg','R_21 selec 2 deg','R_11 tot 2 deg',
                                'R_22 tot 2 deg','R_12 tot 2 deg','R_21 tot 2 deg','psf 1','psf 2','module psf','size gal',
                                'n epoch','mag','size psf','snr','weight gal','peaks','c1 2 deg','c2 2 deg'])

df2_2deg = pd.DataFrame(data,columns=['n gal','kappa_E 2 deg','kappa_B 2 deg','R diag shear 2 deg','R off diag shear 2 deg',
                                 'R diag selec 2 deg','R off diag selec 2 deg','R diag tot 2 deg','R off diag tot 2 deg',
                                'module psf','c1 2 deg','c2 2 deg'])

In [ ]:
corrMatrix = df_2deg.corr()
plt.figure(figsize=(15,15))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=False, annot_kws={"fontsize":8},vmin=-1,vmax=1.,linewidth=0.2)
plt.savefig('./P3X/matrix_corr_2deg.png')
plt.show()

In [ ]:
corrMatrix = df2_2deg.corr()
plt.figure(figsize=(12,12))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=True, annot_kws={"fontsize":9},vmin=-1,vmax=1.)
plt.savefig('./P3X/matrix_corr_reduite_2deg.png',bbox_inches="tight")
plt.show()

In [ ]:
df_4deg = pd.DataFrame(data,columns=['n gal','kappa_E','kappa_E smooth','kappa_B','kappa_B smooth','kappa_E 4 deg',
                                'kappa_B 4 deg','R_11 shear 4 deg','R_22 shear 4 deg','R_12 shear 4 deg','R_21 shear 4 deg',
                                'R_11 selec 4 deg','R_22 selec 4 deg','R_12 selec 4 deg','R_21 selec 4 deg','R_11 tot 4 deg',
                                'R_22 tot 4 deg','R_12 tot 4 deg','R_21 tot 4 deg','psf 1','psf 2','module psf','size gal',
                                'n epoch','mag','size psf','snr','weight gal','peaks','c1 4 deg','c2 4 deg'])

df2_4deg = pd.DataFrame(data,columns=['n gal','kappa_E 4 deg','kappa_B 4 deg','R diag shear 4 deg','R off diag shear 4 deg',
                                 'R diag selec 4 deg','R off diag selec 4 deg','R diag tot 4 deg','R off diag tot 4 deg',
                                'module psf','c1 4 deg','c2 4 deg'])

In [ ]:
corrMatrix = df_4deg.corr()
plt.figure(figsize=(15,15))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=False, annot_kws={"fontsize":8},vmin=-1,vmax=1.,linewidth=0.2)
plt.savefig('./P3X/matrix_corr_4deg.png')
plt.show()

In [ ]:
corrMatrix = df2_4deg.corr()
plt.figure(figsize=(12,12))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=True, annot_kws={"fontsize":9},vmin=-1,vmax=1.)
plt.savefig('./P3X/matrix_corr_reduite_4deg.png',bbox_inches="tight")
plt.show()

In [ ]:
df_05deg = pd.DataFrame(data,columns=['n gal','kappa_E','kappa_E smooth','kappa_B','kappa_B smooth','kappa_E 05 deg',
                                'kappa_B 05 deg','R_11 shear 05 deg','R_22 shear 05 deg','R_12 shear 05 deg','R_21 shear 05 deg',
                                'R_11 selec 05 deg','R_22 selec 05 deg','R_12 selec 05 deg','R_21 selec 05 deg','R_11 tot 05 deg',
                                'R_22 tot 05 deg','R_12 tot 05 deg','R_21 tot 05 deg','psf 1','psf 2','module psf','size gal',
                                'n epoch','mag','size psf','snr','weight gal','peaks','c1 05 deg','c2 05 deg'])

df2_05deg = pd.DataFrame(data,columns=['n gal','kappa_E 05 deg','kappa_B 05 deg','R diag shear 05 deg','R off diag shear 05 deg',
                                 'R diag selec 05 deg','R off diag selec 05 deg','R diag tot 05 deg','R off diag tot 05 deg',
                                'module psf','c1 05 deg','c2 05 deg'])

In [ ]:
corrMatrix = df_05deg.corr()
plt.figure(figsize=(15,15))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=False, annot_kws={"fontsize":8},vmin=-1,vmax=1.,linewidth=0.2)
plt.savefig('./P3X/matrix_corr_05deg.png')
plt.show()

In [ ]:
corrMatrix = df2_05deg.corr()
plt.figure(figsize=(12,12))
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
sns.heatmap(corrMatrix, cmap=new_cmap, annot=True, annot_kws={"fontsize":9},vmin=-1,vmax=1.)
plt.savefig('./P3X/matrix_corr_reduite_05deg.png',bbox_inches="tight")
plt.show()

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rshear_diag_1deg[a]),
            'ngal_Rshear','R shear','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
stats.pearsonr(ngaltot[a],Rshear_diag_1deg[a])

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(tpsf_map[a]),
            'psf','psf','ngal', weights=None,n_bin=100,save_plot=False)

In [ ]:
from scipy import stats
stats.pearsonr(ngaltot[a],tpsf_map[a])

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rshear_diag_2deg[a]),
            'ngal_Rshear','R shear','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
stats.pearsonr(ngaltot[a],Rshear_diag_2deg[a])

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rshear_diag_4deg[a]),
            'ngal_Rshear','R shear','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
stats.pearsonr(ngaltot[a],Rshear_diag_4deg[a])

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rselec_diag_1deg[a]),
            'ngal_Rselec','R selec','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
stats.pearsonr(ngaltot[a],Rselec_diag_1deg[a])

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rselec_diag_2deg[a]),
            'ngal_Rselec','R selec','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rselec_diag_4deg[a]),
            'ngal_Rselec','R selec','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rtot_diag_1deg[a]),
            'ngal_Rtot','R tot','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rtot_diag_2deg[a]),
            'ngal_Rtot','R tot','ngal', weights=None,n_bin=50,save_plot=False)

In [ ]:
psf_e_corr2(np.ravel(ngaltot[a]),np.ravel(Rtot_diag_4deg[a]),
            'ngal_Rtot','R tot','ngal', weights=None,n_bin=50,save_plot=False)

End of local calib

In [ ]:
n_gal_ngmix = len(g_ngmix[0])
d_gal_ngmix = n_gal_ngmix / area_amin2
print("Final number of galaxies : {}\nGalaxy density : {:.2f} gal/arcmin2".format(n_gal_ngmix, d_gal_ngmix))
stat_file.write("Final number of galaxies : {}\nGalaxy density : {:.2f} gal/arcmin2\n".format(n_gal_ngmix, d_gal_ngmix))

In [ ]:
plt.figure(figsize=(30, 30))
plt.hexbin(ra_ngmix, dec_ngmix, gridsize=1000)
cbar = plt.colorbar()
cbar.set_label('Number count', rotation=270)
plt.title("Galaxy number count")
plt.xlabel("Ra (Deg)")
plt.ylabel("Dec (Deg)")
plt.savefig(plot_dir + 'galaxy_number_count_ngmix.png')

In [ ]:
ra_all = dd['XWIN_WORLD']
dec_all = dd['YWIN_WORLD']
plt.figure(figsize=(30, 30))
plt.hexbin(ra_all, dec_all, gridsize=100)
cbar = plt.colorbar()
cbar.set_label('Number count', rotation=270)
plt.title("All objects")
plt.xlabel("Ra (Deg)")
dummy = plt.ylabel("Dec (Deg)")

#### Galsim

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(dd['NGMIX_FLUX_NOSHEAR'][m_gal_ngmix]/dd['NGMIX_FLUX_ERR_NOSHEAR'][m_gal_ngmix], 500, range=(0,200), histtype='step', density=True, label='flux snr ngmix')
plt.hist(dd['SNR_WIN'][m_gal_ngmix], 500, range=(0,200), histtype='step', density=True, label='SEx snr ngmix')
plt.vlines(x=10, ymax=plt.ylim()[1], ymin=0, linestyles='--', colors='k', label='cut : 10')
plt.legend()
plt.xlabel('SNR')
plt.ylabel('frequency')
plt.title('SNR cut')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(dd['SNR_WIN'][m_gal_galsim], 500, range=(0,200), histtype='step', density=True, label='SEx snr galsim')
plt.vlines(x=10, ymax=plt.ylim()[1], ymin=0, linestyles='--', colors='k', label='cut : 10')
plt.legend()
plt.xlabel('SNR')
plt.ylabel('frequency')
dummy = plt.title('SNR cut')

In [ ]:
gal_metacal_galsim = metacal(dd, m_gal_galsim, prefix='GALSIM')

In [ ]:
g_galsim = np.array([gal_metacal_galsim.ns['g1'][gal_metacal_galsim.mask_dict['ns']], gal_metacal_galsim.ns['g2'][gal_metacal_galsim.mask_dict['ns']]])
g_corr_galsim = np.linalg.inv(gal_metacal_galsim.R).dot(g_galsim)
w_galsim = gal_metacal_galsim.ns['w'][gal_metacal_galsim.mask_dict['ns']]
ra_galsim = dd['XWIN_WORLD'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']]
dec_galsim = dd['YWIN_WORLD'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']]
mag_galsim = dd['MAG_AUTO'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']]
#snr_galsim = dd['NGMIX_FLUX_NOSHEAR'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']]/dd['NGMIX_FLUX_ERR_NOSHEAR'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']]

In [ ]:
n_gal_galsim = len(g_galsim[0])
n_tiles = 214 #len(list(set(dd['TILE_ID'])))
area_tiles = 0.25
area_deg = n_tiles*area_tiles
tot_area = area_deg*3600
d_gal_galsim = n_gal_galsim / tot_area
print("Final number of galaxies : {}\nGalaxy density : {:.2f} gal/arcmin2".format(n_gal_galsim, d_gal_galsim))
stat_file.write("Final number of galaxies : {}\nGalaxy density : {:.2f} gal/arcmin2\n".format(n_gal_galsim, d_gal_galsim))

In [ ]:
plt.figure(figsize=(30, 30))
plt.hexbin(ra_galsim, dec_galsim, gridsize=1000)
cbar = plt.colorbar()
cbar.set_label('Number count', rotation=270)
plt.title("Galaxy number count")
plt.xlabel("Ra (Deg)")
plt.ylabel("Dec (Deg)")
plt.savefig(plot_dir + 'galaxy_number_count_galsim.png')

### Compile metacal for stars   
Masking : Flags == 0   &   SNR > 10   &   SNR < 500

#### Ngmix

In [ ]:
star_metacal_ngmix = metacal(dd[ind_star], m_star_ngmix, masking_type='star')

In [ ]:
n_star = len(star_metacal_ngmix.ns['g1'][star_metacal_ngmix.mask_dict['ns']])
tot_area = area_deg*3600
d_star = n_star / tot_area
print("Final number of stars : {}\nStar density : {:.2f}    star/arcmin^2\n               {:.2f} star/deg^2".format(n_star, d_star, d_star*3600))
dummy = stat_file.write("Final number of stars : {}\nStar density : {:.2f}    star/arcmin^2\n               {:.2f} star/deg^2\n".format(n_star, d_star, d_star*3600))

#### Galsim

In [ ]:
star_metacal_galsim = metacal(dd[ind_star], m_star_galsim, masking_type='star')

In [ ]:
n_star = len(star_metacal_galsim.ns['g1'][star_metacal_galsim.mask_dict['ns']])
tot_area = area_deg*3600
d_star = n_star / tot_area
print("Final number of stars : {}\nStar density : {:.2f}    star/arcmin^2\n               {:.2f} star/deg^2".format(n_star, d_star, d_star*3600))
stat_file.write("Final number of stars : {}\nStar density : {:.2f}    star/arcmin^2\n               {:.2f} star/deg^2\n".format(n_star, d_star, d_star*3600))

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(gal_metacal_ngmix.R_shear[0,0], 500, range=(-3,3), histtype='step', density=True, linestyle="-", linewidth=2, color='blue', label='$R11_{gal}$')
plt.hist(gal_metacal_ngmix.R_shear[1,1], 500, range=(-3,3), histtype='step', density=True, linestyle="-", linewidth=2, color='red', label='$R22_{gal}$')
plt.hist(star_metacal_ngmix.R_shear[0,0,:], 500, histtype='step', range=(-3,3), density=True, linestyle=":", linewidth=2, color='blue', label='$R11_{star}$')
plt.hist(star_metacal_ngmix.R_shear[1,1,:], 500, histtype='step', range=(-3,3), density=True, linestyle=":", linewidth=2, color='red', label='$R22_{star}$')
plt.legend()
plt.xlabel('$R_{ii}$')
plt.ylabel('Nb')
plt.title('Response matrix ngmix')
plt.savefig(plot_dir + 'response_matrix_ngmix.png')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(gal_metacal_galsim.R_shear[0,0], 500, range=(-3,3), histtype='step', density=True, linestyle="-", linewidth=2, color='blue', label='$R11_{gal}$')
plt.hist(gal_metacal_galsim.R_shear[1,1], 500, range=(-3,3), histtype='step', density=True, linestyle="-", linewidth=2, color='red', label='$R22_{gal}$')
plt.hist(star_metacal_galsim.R_shear[0,0,:], 500, histtype='step', range=(-3,3), density=True, linestyle=":", linewidth=2, color='blue', label='$R11_{star}$')
plt.hist(star_metacal_galsim.R_shear[1,1,:], 500, histtype='step', range=(-3,3), density=True, linestyle=":", linewidth=2, color='red', label='$R22_{star}$')
plt.legend()
plt.xlabel('$R_{ii}$')
plt.ylabel('Nb')
plt.title('Response matrix galsim')
plt.savefig(plot_dir + 'response_matrix_galsim.png')

### Galaxy response

#### Ngmix

In [ ]:
print('Galaxy total response matrix ngmix : [[{:.2f}    {:.2e}]\n                                [{:.2e}    {:.2f}]]'.format(gal_metacal_ngmix.R[0,0], gal_metacal_ngmix.R[0,1], gal_metacal_ngmix.R[1,0], gal_metacal_ngmix.R[1,1]))
dummy = stat_file.write('Galaxy total response matrix ngmix : [[{:.2f}    {:.2e}]\n                                [{:.2e}    {:.2f}]]\n'.format(gal_metacal_ngmix.R[0,0], gal_metacal_ngmix.R[0,1], gal_metacal_ngmix.R[1,0], gal_metacal_ngmix.R[1,1]))

In [ ]:
print('Galaxy selection response matrix ngmix : [[{:.3f}    {:.2e}]\n                                    [{:.2e}    {:.3f}]]'.format(gal_metacal_ngmix.R_selection[0,0], gal_metacal_ngmix.R_selection[0,1], gal_metacal_ngmix.R_selection[1,0], gal_metacal_ngmix.R_selection[1,1]))
dummy = stat_file.write('Galaxy selection response matrix ngmix : [[{:.3f}    {:.2e}]\n                                    [{:.2e}    {:.3f}]]\n'.format(gal_metacal_ngmix.R_selection[0,0], gal_metacal_ngmix.R_selection[0,1], gal_metacal_ngmix.R_selection[1,0], gal_metacal_ngmix.R_selection[1,1]))

In [ ]:
R_shear = np.mean(gal_metacal_ngmix.R_shear, 2)
print(R_shear)
print(gal_metacal_ngmix.R_selection)
R_sum = R_shear + gal_metacal_ngmix.R_selection
print(R_sum)

In [ ]:
print(np.shape(gal_metacal_ngmix.R_selection))

#### Galsim

In [ ]:
print('Galaxy total response matrix galsim : [[{:.2f}    {:.2e}]\n                                [{:.2e}    {:.2f}]]'.format(gal_metacal_galsim.R[0,0], gal_metacal_galsim.R[0,1], gal_metacal_galsim.R[1,0], gal_metacal_galsim.R[1,1]))
stat_file.write('Galaxy total response matrix galsim : [[{:.2f}    {:.2e}]\n                                [{:.2e}    {:.2f}]]\n'.format(gal_metacal_galsim.R[0,0], gal_metacal_galsim.R[0,1], gal_metacal_galsim.R[1,0], gal_metacal_galsim.R[1,1]))

In [ ]:
print('Galaxy selection response matrix galsim : [[{:.3f}    {:.2e}]\n                                    [{:.2e}    {:.3f}]]'.format(gal_metacal_galsim.R_selection[0,0], gal_metacal_galsim.R_selection[0,1], gal_metacal_galsim.R_selection[1,0], gal_metacal_galsim.R_selection[1,1]))
stat_file.write('Galaxy selection response matrix galsim : [[{:.3f}    {:.2e}]\n                                    [{:.2e}    {:.3f}]]\n'.format(gal_metacal_galsim.R_selection[0,0], gal_metacal_galsim.R_selection[0,1], gal_metacal_galsim.R_selection[1,0], gal_metacal_galsim.R_selection[1,1]))

### Star response

In [ ]:
print('Star total response matrix ngmix : [[{:.2f}     {:.2e}]\n                              [{:.2e}    {:.2f}]]'.format(star_metacal_ngmix.R[0,0], star_metacal_ngmix.R[0,1], star_metacal_ngmix.R[1,0], star_metacal_ngmix.R[1,1]))
stat_file.write('Star total response matrix ngmix : [[{:.2f}     {:.2e}]\n                              [{:.2e}    {:.2f}]]\n'.format(star_metacal_ngmix.R[0,0], star_metacal_ngmix.R[0,1], star_metacal_ngmix.R[1,0], star_metacal_ngmix.R[1,1]))

In [ ]:
print('Star total response matrix galsim : [[{:.2f}     {:.2e}]\n                              [{:.2e}    {:.2f}]]'.format(star_metacal_galsim.R[0,0], star_metacal_galsim.R[0,1], star_metacal_galsim.R[1,0], star_metacal_galsim.R[1,1]))
stat_file.write('Star total response matrix galsim : [[{:.2f}     {:.2e}]\n                              [{:.2e}    {:.2f}]]\n'.format(star_metacal_galsim.R[0,0], star_metacal_galsim.R[0,1], star_metacal_galsim.R[1,0], star_metacal_galsim.R[1,1]))

In [ ]:
# The following creates an eror, star_response not found
#print('Star selection response matrix : [[{:.2e}    {:.2e}]\n                                  [{:.2e}    {:.2e}]]'.format(star_metacal.R_selection[0,0], star_metacal.R_selection[0,1], star_metacal.R_selection[1,0], star_metacal.R_selection[1,1]))
#stat_file.write('Star selection response matrix : [[{:.2e}    {:.2e}]\n                                  [{:.2e}    {:.2e}]]\n'.format(star_metacal.R_selection[0,0], star_metacal.R_selection[0,1], star_metacal.R_selection[1,0], star_metacal.R_selection[1,1]))

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(g_corr_ngmix[0], 500, weights=w_ngmix, histtype='step', density=True, label="$e_{1}$")
plt.hist(g_corr_ngmix[1], 500, weights=w_ngmix, histtype='step', density=True, label="$e_{2}$")
plt.legend()
plt.xlabel('$e_{i}$')
plt.ylabel('Nb')
plt.title("$e_{\\rm gal}$ distribution ngmix")
plt.savefig(plot_dir + 'e_gal_distribution_ngmix.png')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(g_corr_galsim[0], 500, weights=w_galsim, histtype='step', density=True, label="$e_{1}$")
plt.hist(g_corr_galsim[1], 500, weights=w_galsim, histtype='step', density=True, label="$e_{2}$")
plt.legend()
plt.xlabel('$e_{i}$')
plt.ylabel('Nb')
plt.title("$e_{\\rm gal}$ distribution galsim")
plt.savefig(plot_dir + 'e_gal_distribution.png')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(star_metacal_ngmix.ns['g1'][star_metacal_ngmix.mask_dict['ns']], 500, weights=star_metacal_ngmix.ns['w'][star_metacal_ngmix.mask_dict['ns']], histtype='step', density=True, label="$e_{1}$")
plt.hist(star_metacal_ngmix.ns['g2'][star_metacal_ngmix.mask_dict['ns']], 500, weights=star_metacal_ngmix.ns['w'][star_metacal_ngmix.mask_dict['ns']], histtype='step', density=True, label="$e_{2}$")
plt.legend()
plt.xlabel('$e_{i}$')
plt.ylabel('Nb')
plt.title("$e_{star}$ distribution")
plt.savefig(plot_dir + 'e_star_distribution.png')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(star_metacal_galsim.ns['g1'][star_metacal_galsim.mask_dict['ns']], 500, weights=star_metacal_galsim.ns['w'][star_metacal_galsim.mask_dict['ns']], histtype='step', density=True, label="$e_{1}$")
plt.hist(star_metacal_galsim.ns['g2'][star_metacal_galsim.mask_dict['ns']], 500, weights=star_metacal_galsim.ns['w'][star_metacal_galsim.mask_dict['ns']], histtype='step', density=True, label="$e_{2}$")
plt.legend()
plt.xlabel('$e_{i}$')
plt.ylabel('Nb')
plt.title("$e_{star}$ distribution")
plt.savefig(plot_dir + 'e_star_distribution_galsim.png')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(dd['MAG_AUTO'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']], 500)
plt.xlabel('Mag')
plt.ylabel('Nb')
plt.title('Magnitude distribution')
plt.savefig(plot_dir + 'gal_mag_distribution_ngmix.png')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(dd['MAG_AUTO'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']], 500)
plt.xlabel('Mag')
plt.ylabel('Nb')
plt.title('Magnitude distribution')
plt.savefig(plot_dir + 'gal_mag_distribution_galsim.png')

In [ ]:
plt.figure(figsize=(30,20))
plt.hist(dd['MAG_AUTO'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']], 500, weights=w_ngmix, histtype='step', density=True)
plt.hist(dd['MAG_AUTO'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']], 500, histtype='step', density=True)
plt.xlabel('Mag')
plt.ylabel('Nb')
plt.title('Magnitude distribution')
plt.savefig(plot_dir + 'gal_mag_distribution_both.png')

## Additive bias

#### Ngmix

In [ ]:
c1_ngmix, err_c1_ngmix = jackknif_weighted_average(g_corr_ngmix[0], w_ngmix, remove_size=0.05, n_realization=500)
print('Additive bias c1 : {:.3e} +/- {:.3e}'.format(c1_ngmix, err_c1_ngmix))
stat_file.write('Additive bias c1 : {:.3e} +/- {:.3e}\n'.format(c1_ngmix, err_c1_ngmix))

In [ ]:
c2_ngmix, err_c2_ngmix = jackknif_weighted_average(g_corr_ngmix[1], w_ngmix, remove_size=0.05, n_realization=500)
print('Additive bias c2 : {:.3e} +/- {:.3e}'.format(c2_ngmix, err_c2_ngmix))
stat_file.write('Additive bias c2 : {:.3e} +/- {:.3e}\n'.format(c2_ngmix, err_c2_ngmix))

#### Galsim

In [ ]:
c1_galsim, err_c1_galsim = jackknif_weighted_average(g_corr_galsim[0], np.ones_like(w_galsim), remove_size=0.05, n_realization=500)
print('Additive bias c1 : {:.3e} +/- {:.3e}'.format(c1_galsim, err_c1_galsim))
stat_file.write('Additive bias c1 : {:.3e} +/- {:.3e}\n'.format(c1_galsim, err_c1_galsim))

In [ ]:
c2_galsim, err_c2_galsim = jackknif_weighted_average(g_corr_galsim[1], np.ones_like(w_galsim), remove_size=0.05, n_realization=500)
print('Additive bias c2 : {:.3e} +/- {:.3e}'.format(c2_galsim, err_c2_galsim))
stat_file.write('Additive bias c2 : {:.3e} +/- {:.3e}\n'.format(c2_galsim, err_c2_galsim))

In [ ]:
sig_eps = np.sqrt(np.var(g_corr_ngmix[0]) + np.var(g_corr_ngmix[1]))
print('Ellipticity dispersion (complex; mean per component): {:.3f}; {:.3f}'.format(sig_eps, sig_eps /  np.sqrt(2)))

## PSF leakage

### Binning leakage

Use PSF ellipticity at galaxy positions.

#### Ngmix

In [ ]:
psf_e_corr(g_corr_ngmix[0]-c1_ngmix, 
           g_corr_ngmix[1]-c2_ngmix,
           dd['NGMIX_ELL_PSFo_NOSHEAR'][:,0][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']], 
           dd['NGMIX_ELL_PSFo_NOSHEAR'][:,1][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']], 
           dd['NGMIX_T_PSFo_NOSHEAR'][m_gal_ngmix][gal_metacal_ngmix.mask_dict['ns']]/1.17741*2.355, 
           weights=w_ngmix, 
           n_bin=30,
           save_plot=True,
           plot_dir=plot_dir + '/psf_leak_ngmix/')

In [ ]:
psf_e_corr(g_corr_galsim[0]-c1_galsim, 
           g_corr_galsim[1]-c2_galsim,
           dd['GALSIM_PSF_ELL_ORIGINAL_PSF'][:,0][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']], 
           dd['GALSIM_PSF_ELL_ORIGINAL_PSF'][:,1][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']], 
           dd['GALSIM_PSF_SIGMA_ORIGINAL_PSF'][m_gal_galsim][gal_metacal_galsim.mask_dict['ns']]*2.355*0.187, 
           weights=np.ones_like(w_galsim), 
           n_bin=30,
           save_plot=True,
           plot_dir=plot_dir + '/psf_leak_galsim/')

### Alpha leakage

Use PSF ellipticity at star positions.

Equation alpha leakage :
$
    \alpha = \frac{\xi_{+}^{gp} - \langle e_{gal} \rangle^{*} \langle e_{PSF} \rangle}{\xi_{+}^{pp} - |\langle e_{PSF} \rangle|^{2}}
$    
With $g$: galaxy and $p$: PSF.

#### Ngmix

In [ ]:
r_corr_gp_ngmix = xi_star_gal_tc(ra_ngmix, dec_ngmix, g_corr_ngmix[0]-c1_ngmix, g_corr_ngmix[1]-c2_ngmix, w_ngmix, ra_star_ngmix, dec_star_ngmix, g_star_psf_ngmix[0], g_star_psf_ngmix[1])
r_corr_pp_ngmix = xi_star_gal_tc(ra_star_ngmix, dec_star_ngmix, g_star_psf_ngmix[0], g_star_psf_ngmix[1], np.ones_like(g_star_psf_ngmix[0]), ra_star_ngmix, dec_star_ngmix, g_star_psf_ngmix[0], g_star_psf_ngmix[1])

In [ ]:
complex_gal_ngmix = np.average(g_corr_ngmix[0]-c1_ngmix, weights=w_ngmix) + np.average(g_corr_ngmix[1]-c2_ngmix, weights=w_ngmix)*1j
complex_psf_ngmix = np.mean(g_star_psf_ngmix[0]) + np.mean(g_star_psf_ngmix[1])*1j

In [ ]:
alpha_leak_ngmix = (r_corr_gp_ngmix.xip - np.real(np.conj(complex_gal_ngmix)*complex_psf_ngmix))/(r_corr_pp_ngmix.xip - np.abs(complex_psf_ngmix)**2.)

In [ ]:
sig_alpha_leak_ngmix = np.abs(alpha_leak_ngmix)*np.sqrt((np.sqrt(r_corr_gp_ngmix.varxip)/r_corr_gp_ngmix.xip)**2. + (np.sqrt(r_corr_pp_ngmix.varxip)/r_corr_pp_ngmix.xip)**2.)

In [ ]:
alpha_leak_mean_ngmix = np.average(alpha_leak_ngmix, weights=1/sig_alpha_leak_ngmix**2.)
print('All sales weitghed average alpha : {}'.format(alpha_leak_mean_ngmix))
stat_file.write('All scales weitghed average alpha : {}\n'.format(alpha_leak_mean_ngmix))

In [ ]:
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['font.size'] = 35
mpl.rcParams['xtick.minor.size'] = 7
mpl.rcParams['ytick.minor.size'] = 7
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['boxplot.boxprops.linewidth'] = 2.
mpl.rcParams['boxplot.medianprops.linewidth'] = 2.
mpl.rcParams['boxplot.flierprops.markersize'] = 12
mpl.rcParams['boxplot.whiskerprops.linewidth'] = 2.
mpl.rcParams['boxplot.capprops.linewidth'] = 2.


mpl.rcParams['axes.xmargin'] = mpl.rcParamsDefault['axes.xmargin']

plt.figure(figsize=(30,20))

plt.errorbar(r_corr_gp_ngmix.meanr, alpha_leak_ngmix, yerr=sig_alpha_leak_ngmix, label=r'$\alpha$')
plt.hlines(y=0, xmin=plt.xlim()[0], xmax=plt.xlim()[1], linestyles='dashed')

plt.xscale('log')
#plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.5,plt.xlim()[1]-30)
#plt.ylim(-0.5,0.5)
plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\alpha$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.legend()
plt.savefig(plot_dir + 'alpha_leakage_ngmix.png')

## $\xi^{sys}$

Use PSF ellipticity at star positions.

Equation alpha leakage :
$
\xi^{sys} = \frac{\xi_{\pm}^{gp^{2}}}{\xi_{\pm}^{pp}}
$    
With $g$: galaxy and $p$: PSF.

In [ ]:
C_sys_p_ngmix = r_corr_gp_ngmix.xip**2/r_corr_pp_ngmix.xip
C_sys_m_ngmix = r_corr_gp_ngmix.xim**2/r_corr_pp_ngmix.xim
#C_sys_var = np.sqrt(r_corr_gp.varxi**2./r_corr_pp.varxi)
C_sys_std_p_ngmix = np.abs(C_sys_p_ngmix) * np.sqrt((((2*r_corr_gp_ngmix.xip**2.*np.sqrt(r_corr_gp_ngmix.varxip))/r_corr_gp_ngmix.xip)/r_corr_gp_ngmix.xip**2.)**2. + (np.sqrt(r_corr_pp_ngmix.varxip)/r_corr_pp_ngmix.xip)**2.)
C_sys_std_m_ngmix = np.abs(C_sys_m_ngmix) * np.sqrt((((2*r_corr_gp_ngmix.xim**2.*np.sqrt(r_corr_gp_ngmix.varxim))/r_corr_gp_ngmix.xim)/r_corr_gp_ngmix.xim**2.)**2. + (np.sqrt(r_corr_pp_ngmix.varxim)/r_corr_pp_ngmix.xim)**2.)

## Redshift distribution

In [ ]:
nz_base = 'nz.CFHTLenSmatched.W3'
nz_ext = 'txt'
nz_date = '202012'
nz_version = 'v1'
nz_name = '{}_{}_{}.{}'.format(nz_base, nz_date, nz_version, nz_ext)
source = 'vos:cfis/cosmostat/cosmology/redshifts/{}'.format(nz_name)

download(source, nz_name, verbose=True)

In [ ]:
z, nz = np.loadtxt(nz_name, unpack=True)

#theta = np.logspace(np.log10(2), np.log10(2e2), 20)

Om = 0.3153
sig8 = 0.8111
ns = 0.9649
Ob = 0.0493
h = 0.6736
xi_p_planck, xi_m_planck = get_theo_xi(r_corr_gp_ngmix.meanr, z, nz, Omega_m=Om, h=h, Omega_b=Ob, sig8=sig8, ns=ns)

In [ ]:
#mpl.rcParams['lines.linewidth'] = 3
#mpl.rcParams['lines.markersize'] = 10
#mpl.rcParams['font.size'] = 35
#mpl.rcParams['xtick.minor.size'] = 7
#mpl.rcParams['ytick.minor.size'] = 7
#mpl.rcParams['xtick.major.size'] = 10
#mpl.rcParams['ytick.major.size'] = 10
#mpl.rcParams['xtick.major.width'] = 3
#mpl.rcParams['ytick.major.width'] = 3
#mpl.rcParams['boxplot.boxprops.linewidth'] = 2.
#mpl.rcParams['boxplot.medianprops.linewidth'] = 2.
#mpl.rcParams['boxplot.flierprops.markersize'] = 12
#mpl.rcParams['boxplot.whiskerprops.linewidth'] = 2.
#mpl.rcParams['boxplot.capprops.linewidth'] = 2.
#mpl.rcParams['axes.xmargin'] = mpl.rcParamsDefault['axes.xmargin']

color = plt.rcParams['axes.prop_cycle'].by_key()['color']

plt.figure(figsize=(30,20))
plt.errorbar(r_corr_gp_ngmix.meanr, C_sys_p_ngmix, yerr=C_sys_std_p_ngmix, label=r'$\xi_{+}^{sys}$')
plt.plot(r_corr_gp_ngmix.meanr, xi_p_planck, ':', color=color[0], label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$')
plt.plot(r_corr_gp_ngmix.meanr, xi_m_planck, ':', color=color[1], label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$')

#ind_neg = np.where(C_sys_m <= 0)[0]
#ind_pos = np.where(C_sys_m > 0)[0]
#plt.errorbar(res.meanr[ind_neg]+0.05*res.meanr[ind_neg], np.abs(C_sys_m[ind_neg]), yerr=C_sys_std_m[ind_neg], linestyle='--', color=color[1], label=r'$\xi_{-}^{sys}$')
#plt.errorbar(res.meanr[[ind_neg[-1], ind_pos[0]]]+0.05*res.meanr[[ind_neg[-1], ind_pos[0]]], np.abs(C_sys_m[[ind_neg[-1], ind_pos[0]]]), yerr=C_sys_std_m[[ind_neg[-1], ind_pos[0]]], linestyle='--', color=color[1])
#plt.errorbar(res.meanr[ind_pos]+0.05*res.meanr[ind_pos], C_sys_m[ind_pos], yerr=C_sys_std_m[ind_pos], color=color[1], linestyle='-')
plt.errorbar(r_corr_gp_ngmix.meanr*(1.02), C_sys_m_ngmix, yerr=C_sys_std_m_ngmix, label=r'$\xi_{-}^{sys}$')

plt.xscale('log')
plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.,plt.xlim()[1]-30)

plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{\pm}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_sys_ngmix.png')

In [ ]:
#mpl.rcParams['lines.linewidth'] = 3
#mpl.rcParams['lines.markersize'] = 10
#mpl.rcParams['font.size'] = 35
#mpl.rcParams['xtick.minor.size'] = 7
#mpl.rcParams['ytick.minor.size'] = 7
#mpl.rcParams['xtick.major.size'] = 10
#mpl.rcParams['ytick.major.size'] = 10
#mpl.rcParams['xtick.major.width'] = 3
#mpl.rcParams['ytick.major.width'] = 3
#mpl.rcParams['boxplot.boxprops.linewidth'] = 2.
#mpl.rcParams['boxplot.medianprops.linewidth'] = 2.
#mpl.rcParams['boxplot.flierprops.markersize'] = 12
#mpl.rcParams['boxplot.whiskerprops.linewidth'] = 2.
#mpl.rcParams['boxplot.capprops.linewidth'] = 2.
#mpl.rcParams['axes.xmargin'] = mpl.rcParamsDefault['axes.xmargin']

color = plt.rcParams['axes.prop_cycle'].by_key()['color']

plt.figure(figsize=(30,20))
#plt.errorbar(r_corr_gp.meanr, C_sys_p, yerr=C_sys_std_p, label=r'$\xi_{+}^{sys}$')
#plt.plot(r_corr_gp.meanr, f_theo_p_p(r_corr_gp.meanr), ':', color=color[0], label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$')
#plt.plot(r_corr_gp.meanr, f_theo_m_p(r_corr_gp.meanr), ':', color=color[1], label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$')
plt.plot(r_corr_gp_ngmix.meanr, C_sys_p_ngmix/xi_p_planck, color=color[0], label=r'$\xi_{\pm}/\xi_{+}^{\Lambda\mathrm{CDM}}$')

#ind_neg = np.where(C_sys_m <= 0)[0]
#ind_pos = np.where(C_sys_m > 0)[0]
#plt.errorbar(res.meanr[ind_neg]+0.05*res.meanr[ind_neg], np.abs(C_sys_m[ind_neg]), yerr=C_sys_std_m[ind_neg], linestyle='--', color=color[1], label=r'$\xi_{-}^{sys}$')
#plt.errorbar(res.meanr[[ind_neg[-1], ind_pos[0]]]+0.05*res.meanr[[ind_neg[-1], ind_pos[0]]], np.abs(C_sys_m[[ind_neg[-1], ind_pos[0]]]), yerr=C_sys_std_m[[ind_neg[-1], ind_pos[0]]], linestyle='--', color=color[1])
#plt.errorbar(res.meanr[ind_pos]+0.05*res.meanr[ind_pos], C_sys_m[ind_pos], yerr=C_sys_std_m[ind_pos], color=color[1], linestyle='-')
#plt.errorbar(r_corr_gp.meanr*(1.02), C_sys_m, yerr=C_sys_std_m, label=r'$\xi_{-}^{sys}$')

plt.xscale('log')
#plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.,plt.xlim()[1]-30)

plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{\pm}/\xi_{+}^{\Lambda\mathrm{CDM}}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_sys_ratio_ngmix.png')

#### Galsim

In [ ]:
r_corr_gp_galsim = xi_star_gal_tc(ra_galsim, dec_galsim, g_corr_galsim[0]-c1_galsim, g_corr_galsim[1]-c2_galsim, np.ones_like(w_galsim), ra_star_galsim, dec_star_galsim, g_star_psf_galsim[0], g_star_psf_galsim[1])
r_corr_pp_galsim = xi_star_gal_tc(ra_star_galsim, dec_star_galsim, g_star_psf_galsim[0], g_star_psf_galsim[1], np.ones_like(g_star_psf_galsim[0]), ra_star_galsim, dec_star_galsim, g_star_psf_galsim[0], g_star_psf_galsim[1])

In [ ]:
complex_gal_galsim = np.average(g_corr_galsim[0]-c1_galsim, weights=np.ones_like(w_galsim)) + np.average(g_corr_galsim[1]-c2_galsim, weights=np.ones_like(w_galsim))*1j
complex_psf_galsim = np.mean(g_star_psf_galsim[0]) + np.mean(g_star_psf_galsim[1])*1j

In [ ]:
alpha_leak_galsim = (r_corr_gp_galsim.xip - np.real(np.conj(complex_gal_galsim)*complex_psf_galsim))/(r_corr_pp_galsim.xip - np.abs(complex_psf_galsim)**2.)

In [ ]:
sig_alpha_leak_galsim = np.abs(alpha_leak_galsim)*np.sqrt((np.sqrt(r_corr_gp_galsim.varxip)/r_corr_gp_galsim.xip)**2. + (np.sqrt(r_corr_pp_galsim.varxip)/r_corr_pp_galsim.xip)**2.)

In [ ]:
alpha_leak_mean_galsim = np.average(alpha_leak_galsim, weights=1/sig_alpha_leak_galsim**2.)
print('All sales weitghed average alpha : {}'.format(alpha_leak_mean_galsim))
stat_file.write('All sales weitghed average alpha : {}\n'.format(alpha_leak_mean_galsim))

In [ ]:
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['font.size'] = 35
mpl.rcParams['xtick.minor.size'] = 7
mpl.rcParams['ytick.minor.size'] = 7
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['boxplot.boxprops.linewidth'] = 2.
mpl.rcParams['boxplot.medianprops.linewidth'] = 2.
mpl.rcParams['boxplot.flierprops.markersize'] = 12
mpl.rcParams['boxplot.whiskerprops.linewidth'] = 2.
mpl.rcParams['boxplot.capprops.linewidth'] = 2.


mpl.rcParams['axes.xmargin'] = mpl.rcParamsDefault['axes.xmargin']

plt.figure(figsize=(30,20))

plt.errorbar(r_corr_gp_galsim.meanr, alpha_leak_galsim, yerr=sig_alpha_leak_galsim, label=r'$\alpha$')
plt.hlines(y=0, xmin=plt.xlim()[0], xmax=plt.xlim()[1], linestyles='dashed')

plt.xscale('log')
#plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.5,plt.xlim()[1]-30)
#plt.ylim(-0.5,0.5)
plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\alpha$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.legend()
plt.savefig(plot_dir + 'alpha_leakage_galsim.png')

In [ ]:
C_sys_p_galsim = r_corr_gp_galsim.xip**2/r_corr_pp_galsim.xip
C_sys_m_galsim = r_corr_gp_galsim.xim**2/r_corr_pp_galsim.xim
#C_sys_var = np.sqrt(r_corr_gp.varxi**2./r_corr_pp.varxi)
C_sys_std_p_galsim = np.abs(C_sys_p_galsim) * np.sqrt((((2*r_corr_gp_galsim.xip**2.*np.sqrt(r_corr_gp_galsim.varxip))/r_corr_gp_galsim.xip)/r_corr_gp_galsim.xip**2.)**2. + (np.sqrt(r_corr_pp_galsim.varxip)/r_corr_pp_galsim.xip)**2.)
C_sys_std_m_galsim = np.abs(C_sys_m_galsim) * np.sqrt((((2*r_corr_gp_galsim.xim**2.*np.sqrt(r_corr_gp_galsim.varxim))/r_corr_gp_galsim.xim)/r_corr_gp_galsim.xim**2.)**2. + (np.sqrt(r_corr_pp_galsim.varxim)/r_corr_pp_galsim.xim)**2.)

In [ ]:
#mpl.rcParams['lines.linewidth'] = 3
#mpl.rcParams['lines.markersize'] = 10
#mpl.rcParams['font.size'] = 35
#mpl.rcParams['xtick.minor.size'] = 7
#mpl.rcParams['ytick.minor.size'] = 7
#mpl.rcParams['xtick.major.size'] = 10
#mpl.rcParams['ytick.major.size'] = 10
#mpl.rcParams['xtick.major.width'] = 3
#mpl.rcParams['ytick.major.width'] = 3
#mpl.rcParams['boxplot.boxprops.linewidth'] = 2.
#mpl.rcParams['boxplot.medianprops.linewidth'] = 2.
#mpl.rcParams['boxplot.flierprops.markersize'] = 12
#mpl.rcParams['boxplot.whiskerprops.linewidth'] = 2.
#mpl.rcParams['boxplot.capprops.linewidth'] = 2.
#mpl.rcParams['axes.xmargin'] = mpl.rcParamsDefault['axes.xmargin']

color = plt.rcParams['axes.prop_cycle'].by_key()['color']

plt.figure(figsize=(30,20))
plt.errorbar(r_corr_gp_galsim.meanr, C_sys_p_galsim, yerr=C_sys_std_p_galsim, label=r'$\xi_{+}^{sys}$')
plt.plot(r_corr_gp_galsim.meanr, xi_p_planck, ':', color=color[0], label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$')
plt.plot(r_corr_gp_galsim.meanr, xi_m_planck, ':', color=color[1], label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$')

#ind_neg = np.where(C_sys_m <= 0)[0]
#ind_pos = np.where(C_sys_m > 0)[0]
#plt.errorbar(res.meanr[ind_neg]+0.05*res.meanr[ind_neg], np.abs(C_sys_m[ind_neg]), yerr=C_sys_std_m[ind_neg], linestyle='--', color=color[1], label=r'$\xi_{-}^{sys}$')
#plt.errorbar(res.meanr[[ind_neg[-1], ind_pos[0]]]+0.05*res.meanr[[ind_neg[-1], ind_pos[0]]], np.abs(C_sys_m[[ind_neg[-1], ind_pos[0]]]), yerr=C_sys_std_m[[ind_neg[-1], ind_pos[0]]], linestyle='--', color=color[1])
#plt.errorbar(res.meanr[ind_pos]+0.05*res.meanr[ind_pos], C_sys_m[ind_pos], yerr=C_sys_std_m[ind_pos], color=color[1], linestyle='-')
plt.errorbar(r_corr_gp_galsim.meanr*(1.02), C_sys_m_galsim, yerr=C_sys_std_m_galsim, label=r'$\xi_{-}^{sys}$')

plt.xscale('log')
plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.,plt.xlim()[1]-30)

plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{\pm}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_sys_galsim.png')

In [ ]:
#mpl.rcParams['lines.linewidth'] = 3
#mpl.rcParams['lines.markersize'] = 10
#mpl.rcParams['font.size'] = 35
#mpl.rcParams['xtick.minor.size'] = 7
#mpl.rcParams['ytick.minor.size'] = 7
#mpl.rcParams['xtick.major.size'] = 10
#mpl.rcParams['ytick.major.size'] = 10
#mpl.rcParams['xtick.major.width'] = 3
#mpl.rcParams['ytick.major.width'] = 3
#mpl.rcParams['boxplot.boxprops.linewidth'] = 2.
#mpl.rcParams['boxplot.medianprops.linewidth'] = 2.
#mpl.rcParams['boxplot.flierprops.markersize'] = 12
#mpl.rcParams['boxplot.whiskerprops.linewidth'] = 2.
#mpl.rcParams['boxplot.capprops.linewidth'] = 2.
#mpl.rcParams['axes.xmargin'] = mpl.rcParamsDefault['axes.xmargin']

color = plt.rcParams['axes.prop_cycle'].by_key()['color']

plt.figure(figsize=(30,20))
#plt.errorbar(r_corr_gp.meanr, C_sys_p, yerr=C_sys_std_p, label=r'$\xi_{+}^{sys}$')
#plt.plot(r_corr_gp.meanr, f_theo_p_p(r_corr_gp.meanr), ':', color=color[0], label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$')
#plt.plot(r_corr_gp.meanr, f_theo_m_p(r_corr_gp.meanr), ':', color=color[1], label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$')
plt.plot(r_corr_gp_galsim.meanr, C_sys_p_galsim/xi_p_planck, color=color[0], label=r'$\xi_{\pm}/\xi_{+}^{\Lambda\mathrm{CDM}}$')

#ind_neg = np.where(C_sys_m <= 0)[0]
#ind_pos = np.where(C_sys_m > 0)[0]
#plt.errorbar(res.meanr[ind_neg]+0.05*res.meanr[ind_neg], np.abs(C_sys_m[ind_neg]), yerr=C_sys_std_m[ind_neg], linestyle='--', color=color[1], label=r'$\xi_{-}^{sys}$')
#plt.errorbar(res.meanr[[ind_neg[-1], ind_pos[0]]]+0.05*res.meanr[[ind_neg[-1], ind_pos[0]]], np.abs(C_sys_m[[ind_neg[-1], ind_pos[0]]]), yerr=C_sys_std_m[[ind_neg[-1], ind_pos[0]]], linestyle='--', color=color[1])
#plt.errorbar(res.meanr[ind_pos]+0.05*res.meanr[ind_pos], C_sys_m[ind_pos], yerr=C_sys_std_m[ind_pos], color=color[1], linestyle='-')
#plt.errorbar(r_corr_gp.meanr*(1.02), C_sys_m, yerr=C_sys_std_m, label=r'$\xi_{-}^{sys}$')

plt.xscale('log')
#plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.,plt.xlim()[1]-30)

plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{\pm}/\xi_{+}^{\Lambda\mathrm{CDM}}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_sys_ratio_galsim.png')

## Write catalog

In [ ]:
# TODO: move this function to the python script.

def write_catalog_bias(output_path, ra, dec, g1, g2, w, mag, snr,
                       R, R_select, c1, c2, c1_err, c2_err, alpha_leak,
                       g1_uncal=None, g2_uncal=None, R_11=None,
                       R_22=None, R_12=None, R_21=None):
    """
    """

    # Data HDU
    c_ra = fits.Column(name='ra', array=ra, format='D', unit='deg')
    c_dec = fits.Column(name='dec', array=dec, format='D', unit='deg')
    c_g1 = fits.Column(name='g1', array=g1, format='D')
    c_g2 = fits.Column(name='g2', array=g2, format='D')
    c_w = fits.Column(name='w', array=w, format='D')
    c_mag = fits.Column(name='mag', array=mag, format='D')
    c_snr = fits.Column(name='snr', array=snr, format='D')
    cols = [c_ra, c_dec, c_g1, c_g2, c_w, c_mag, c_snr]

    for x, name in zip([g1_uncal, g2_uncal, R_11, R_22, R_12, R_21],
                       ['g1_uncal', 'g2_uncal', 'R_11', 'R_22', 'R_12', 'R_21']):
        if x is not None:
            cols.append(fits.Column(name=name, array=x, format='D'))

    table_hdu = fits.BinTableHDU.from_columns(cols)

    table_hdu.header['TTYPE3'] = ('g1', 'Reduced shear (a-b)/(a+b)')
    table_hdu.header['TTYPE4'] = ('g2', 'Reduced shear (a-b)/(a+b)')
    table_hdu.header['TTYPE5'] = ('w', 'Ellipticity weight')
    table_hdu.header['TTYPE6'] = ('mag', 'From SExtractor: MAG_AUTO')
    table_hdu.header['TTYPE7'] = ('snr', 'SNR computed on metacal noshear: flux/flux_std')
    
    ntype =  8
    for x in ([g1_uncal, g2_uncal], ['g1_uncal', 'g2_uncal']):
        if x is not None:
            table_hdu.header['TTYPE{}'.format(ntype)] = (name, 'uncalibrated reduced shear')
            ntype += 1
    for x, name in zip([R_11, R_22, R_12, R_21], ['R_11', 'R_22', 'R_12', 'R_21']):
        if x is not None:
            table_hdu.header['TTYPE{}'.format(ntype)] = (name, f'full response matrix {name}')
            ntype += 1

    # Primary HDU with information in header
    primary_header = fits.Header()

    primary_header['R'] = (r'<R>', r'Full response: <R> = <R_shear> + <R_select>')

    primary_header['R1_1'] = (R[0,0], 'Full response matrix')
    primary_header['R1_2'] = (R[0,1], 'Full response matrix')
    primary_header['R2_1'] = (R[1,0], 'Full response matrix')
    primary_header['R2_2'] = (R[1,1], 'Full response matrix')

    primary_header['RS'] = (r'<RS>', r'Selection response matrix: <R_select>')
    primary_header['RS1_1'] = (R_select[0,0], 'Selection response matrix')
    primary_header['RS1_2'] = (R_select[0,1], 'Selection response matrix')
    primary_header['RS2_1'] = (R_select[1,0], 'Selection response matrix')
    primary_header['RS2_2'] = (R_select[1,1], 'Selection response matrix')

    primary_header['g_corr'] = ('Apply correction', r'g_corr = R**(-1).g')

    primary_header['c'] = ('', 'Additive bias computed with JackKnife')
    primary_header['c1'] = (c1, 'Additive bias for g1')
    primary_header['c1_err'] = (c1_err, 'Standard deviation for c1')
    primary_header['c2'] = (c2, 'Additive bias for g2')
    primary_header['c2_err'] = (c2_err, 'Standard deviation for c2')

    primary_header['w'] = ('Ellipticity weight', r'1 / (2*sig_SN**2 + g1_var + g2_var)')
    primary_header['sig_SN'] = (0.34, 'Sigma_shape_noise')

    primary_header['alpha'] = (alpha_leak, 'PSF leakage alpha')

    primary_hdu = fits.PrimaryHDU(header=primary_header)

    # Final file
    hdu_list = fits.HDUList([primary_hdu, table_hdu])

    hdu_list.writeto(output_path, overwrite=True)


In [ ]:
out_name = 'shape_catalog_W3_MCCD_20210326.fits'

R = np.zeros_like(gal_metacal_ngmix.R_shear)
for i in (0,1):
    for j in (0, 1):
        R[i, j] = gal_metacal_ngmix.R_shear[i, j] + gal_metacal_ngmix.R_selection[i, j]

# Debug
alpha_leaka_mean_ngmix = 0
        
write_catalog_bias(out_name,
                   ra_ngmix, dec_ngmix,
                   g_corr_ngmix[0], g_corr_ngmix[1], w_ngmix,
                   mag_ngmix, snr_ngmix, 
                   gal_metacal_ngmix.R, gal_metacal_ngmix.R_selection,
                   c1_ngmix, c2_ngmix, err_c1_ngmix, err_c2_ngmix,
                   alpha_leaka_mean_ngmix,
                   g1_uncal=g_ngmix[0],g2_uncal=g_ngmix[1], 
                   R_11=R[0, 0], R_12=R[0, 1], R_21=R[1, 0], R_22=R[1, 1])

## Some cosmological calculations

### Ngmix

### Shear-Shear

In [ ]:
res_g_ngmix = xi_gal_gal_tc(ra_ngmix, dec_ngmix, g_corr_ngmix[0]-c1_ngmix, g_corr_ngmix[1]-c2_ngmix, w_ngmix, ra_ngmix, dec_ngmix, g_corr_ngmix[0]-c1_ngmix, g_corr_ngmix[1]-c2_ngmix, w_ngmix)

##### Cosmo planck

In [ ]:
z, nz = np.loadtxt(nz_name, unpack=True)

Om = 0.3153
sig8 = 0.8111
ns = 0.9649
Ob = 0.0493
h = 0.6736
xi_p_planck, xi_m_planck = get_theo_xi(res_g_ngmix.meanr, z, nz, Omega_m=Om, h=h, Omega_b=Ob, sig8=sig8, ns=ns)

##### Cosmo DES

In [ ]:
Om = 0.295
S8 = 0.768
sig8 = S8 / ((Om/0.3)**0.5)
#sig8 = 0.8111
ns = 1.044
Ob = 0.0516
h = 0.672
xi_p_des, xi_m_des = get_theo_xi(res_g_ngmix.meanr, z, nz, Omega_m=Om, h=h, Omega_b=Ob, sig8=sig8, ns=ns)

In [ ]:
pos_ind_gg = res_g_ngmix.xip >= 0
neg_ind_gg = res_g_ngmix.xip < 0

plt.figure(figsize=(15,10))

eb1 = plt.errorbar(res_g_ngmix.meanr[pos_ind_gg], res_g_ngmix.xip[pos_ind_gg], yerr=np.sqrt(res_g_ngmix.varxip[pos_ind_gg]), linestyle='', color=color[0], marker='x', capsize=4, label=r'$\xi_{+}$')
eb1[-1][0].set_linestyle('-')

eb2 = plt.errorbar(res_g_ngmix.meanr[neg_ind_gg], np.abs(res_g_ngmix.xip[neg_ind_gg]), yerr=np.sqrt(res_g_ngmix.varxip[neg_ind_gg]), linestyle='', color=color[0], marker='x', capsize=4)#, label=r'$\xi_{+}$')
eb2[-1][0].set_linestyle(':')

plt.plot(res_g_ngmix.meanr, xi_p_planck, 'k:', label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$ planck')
plt.plot(res_g_ngmix.meanr, xi_p_des, 'k--', label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$ DESY1')

plt.xscale('log')
plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.3,plt.xlim()[1]-0)

plt.xticks([2, 10, 100], labels=['2', '10', '100'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{+}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_+_shear_shear_ngmix.png')

In [ ]:
pos_ind_gg = res_g_ngmix.xim >= 0
neg_ind_gg = res_g_ngmix.xim < 0

plt.figure(figsize=(15,10))

eb1 = plt.errorbar(res_g_ngmix.meanr[pos_ind_gg], res_g_ngmix.xim[pos_ind_gg], yerr=np.sqrt(res_g_ngmix.varxim[pos_ind_gg]), linestyle='', color=color[1], marker='o', markerfacecolor='none', capsize=4, label=r'$\xi_{-}$')
eb1[-1][0].set_linestyle('-')

eb2 = plt.errorbar(res_g_ngmix.meanr[neg_ind_gg], np.abs(res_g_ngmix.xim[neg_ind_gg]), yerr=np.sqrt(res_g_ngmix.varxim[neg_ind_gg]), linestyle='', color=color[1], marker='o', markerfacecolor='none', capsize=4)#, label=r'$\xi_{-}$')
eb2[-1][0].set_linestyle(':')

plt.plot(res_g_ngmix.meanr, xi_m_planck, 'k:', label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$ planck')
plt.plot(res_g_ngmix.meanr, xi_m_des, 'k--', label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$ DESY1')

plt.xscale('log')
plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.,plt.xlim()[1])

plt.xticks([2, 10, 100], labels=['2', '10', '100'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{-}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_-_shear_shear_ngmix.png')

### Galsim

In [ ]:
res_g_galsim = xi_gal_gal_tc(ra_galsim, dec_galsim, g_corr_galsim[0]-c1_galsim, g_corr_galsim[1]-c2_galsim, np.ones_like(w_galsim), ra_galsim, dec_galsim, g_corr_galsim[0]-c1_galsim, g_corr_galsim[1]-c2_galsim, np.ones_like(w_galsim))

In [ ]:
pos_ind_gg = res_g_galsim.xip >= 0
neg_ind_gg = res_g_galsim.xip < 0

plt.figure(figsize=(15,10))

eb1 = plt.errorbar(res_g_galsim.meanr[pos_ind_gg], res_g_galsim.xip[pos_ind_gg], yerr=np.sqrt(res_g_galsim.varxip[pos_ind_gg]), linestyle='', color=color[0], marker='x', capsize=4, label=r'$\xi_{+}$')
eb1[-1][0].set_linestyle('-')

eb2 = plt.errorbar(res_g_galsim.meanr[neg_ind_gg], np.abs(res_g_galsim.xip[neg_ind_gg]), yerr=np.sqrt(res_g_galsim.varxip[neg_ind_gg]), linestyle='', color=color[0], marker='x', capsize=4)#, label=r'$\xi_{+}$')
eb2[-1][0].set_linestyle(':')

plt.plot(res_g_galsim.meanr, xi_p_planck, 'k:', label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$ planck')
plt.plot(res_g_galsim.meanr, xi_p_des, 'k--', label=r'$\xi_{+}^{\Lambda\mathrm{CDM}}$ DESY1')

plt.xscale('log')
plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.,plt.xlim()[1]-0)

plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{+}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_+_shear_shear_galsim.png')

In [ ]:
pos_ind_gg = res_g_galsim.xim >= 0
neg_ind_gg = res_g_galsim.xim < 0

plt.figure(figsize=(15,10))

eb1 = plt.errorbar(res_g_galsim.meanr[pos_ind_gg], res_g_galsim.xim[pos_ind_gg], yerr=np.sqrt(res_g_galsim.varxim[pos_ind_gg]), linestyle='', color=color[1], marker='o', markerfacecolor='none', capsize=4, label=r'$\xi_{-}$')
eb1[-1][0].set_linestyle('-')

eb2 = plt.errorbar(res_g_galsim.meanr[neg_ind_gg], np.abs(res_g_galsim.xim[neg_ind_gg]), yerr=np.sqrt(res_g_galsim.varxim[neg_ind_gg]), linestyle='', color=color[1], marker='o', markerfacecolor='none', capsize=4)#, label=r'$\xi_{-}$')
eb2[-1][0].set_linestyle(':')

plt.plot(res_g_galsim.meanr, xi_m_planck, 'k:', label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$ planck')
plt.plot(res_g_galsim.meanr, xi_m_des, 'k--', label=r'$\xi_{-}^{\Lambda\mathrm{CDM}}$ DESY1')

plt.xscale('log')
plt.yscale('log')
plt.xlim(plt.xlim()[0]+0.,plt.xlim()[1])

plt.xticks([2, 10, 200], labels=['2', '10', '200'])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\xi_{-}$')
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda v,_: ("$10^{%d}$" % math.log(v,10)) ))
plt.legend()
plt.savefig(plot_dir + 'xi_-_shear_shear_galsim.png')

### Cluster lensing

#### Load Planck cluster catalog

In [ ]:
cluster_cat_name = 'HFI_PCCS_SZ-union_R2.08.fits.gz'
source = 'vos:cfis/cosmostat/cosmology/external/Planck/{}'.format(cluster_cat_name)
download(source, cluster_cat_name, verbose=True)

In [ ]:
cluster_cat = fits.getdata(cluster_cat_name)
m_good_cluster = (cluster_cat['MSZ'] != 0) & (cluster_cat['COSMO'] == True)
m_cluster_foot = get_mask_footprint_P3(cluster_cat['RA'][m_good_cluster], cluster_cat['DEC'][m_good_cluster])
sdss_cluster_cut = {'ra': cluster_cat['RA'][m_good_cluster][m_cluster_foot], 'dec':cluster_cat['DEC'][m_good_cluster][m_cluster_foot], 'z':cluster_cat['REDSHIFT'][m_good_cluster][m_cluster_foot], 'M':cluster_cat['MSZ'][m_good_cluster][m_cluster_foot]*1e14}
#sdss_cluster_cut = {'ra': cluster_cat['RA'][m_good_cluster][m_cluster_foot][1:2], 'dec':cluster_cat['DEC'][m_good_cluster][m_cluster_foot][1:2], 'z':cluster_cat['REDSHIFT'][m_good_cluster][m_cluster_foot][1:2], 'M':cluster_cat['MSZ'][m_good_cluster][m_cluster_foot][1:2]*1e14}

In [ ]:
plt.figure(figsize=(15,15))
plt.plot(ra_ngmix, dec_ngmix, '+')
plt.plot(sdss_cluster_cut['ra'], sdss_cluster_cut['dec'], '*')
for i in range(len(sdss_cluster_cut['ra'])):
    plt.text(sdss_cluster_cut['ra'][i], sdss_cluster_cut['dec'][i]+ 0.02 * (plt.ylim()[0] - plt.ylim()[1]), i, color='k', fontsize=10, ha='center', va='center')

In [ ]:
ind_cluster_keep = [2,4,5,6,7,8,9,10,11,13,14,15]
for key in sdss_cluster_cut.keys():
    sdss_cluster_cut[key] = sdss_cluster_cut[key][ind_cluster_keep]

In [ ]:
print('Number of clusters : {}'.format(len(sdss_cluster_cut['z'])))
stat_file.write('Number of clusters : {}\n'.format(len(sdss_cluster_cut['z'])))

In [ ]:
R_p, logR_p, gamT_p, gamX_p, gam_sig_p = gamma_T_tc(sdss_cluster_cut['ra'], sdss_cluster_cut['dec'], ra_ngmix, dec_ngmix, g_corr_ngmix[0]-c1_ngmix, g_corr_ngmix[1]-c2_ngmix, w_ngmix)

In [ ]:
gamT_p

In [ ]:


plt.figure(figsize=(30,20))
# plt.errorbar(logR_p, gamT_p, yerr=gam_sig_p, capsize = 3, label='gamma_T')
# plt.boxplot(res[:,2,:], positions=logR_p, widths=0.2)

errplot = plt.errorbar(logR_p, gamT_p, yerr=gam_sig_p, capsize = 3, label=r'$\gamma_{t}$ known clusters')
#plt.boxplot(res[:,2,:], positions=logR_p, widths=0.2)
plt.hlines(y=0, xmin=np.min(logR_p), xmax=np.max(logR_p), linestyles='dashed')
# plt.xscale('log')
plt.title(r'Lensing by clusters')
plt.ylabel(r'$\gamma$')
plt.xlabel(r'$log(\theta$) (arcmin)')
#plt.xticks(fontsize=20)
#plt.yticks(fontsize=20)

plt.gca().xaxis.set_major_locator(mpl.ticker.MultipleLocator(0.5))
plt.gca().xaxis.set_major_formatter(ScalarFormatter())

plt.xlim(plt.xlim()[0]+0.3,plt.xlim()[1]-0.3)

plt.legend([errplot, Patch(facecolor='w', edgecolor='k')], [r'$\gamma_{t}$ known clusters', r'random positions'])
#plt.savefig('new_cat/cluster_profile_bp.png')

In [ ]:
stacked_gammaT_theo = np.zeros(15)
for m_clust, z_clust in tqdm(zip(sdss_cluster_cut['M'], sdss_cluster_cut['z']), total=len(sdss_cluster_cut['M'])):
    stacked_gammaT_theo += get_theo_gamT(m_clust, z_clust, z_source=0.68, theta=R_p, meanbin_z=z, nz=nz)

In [ ]:
plt.figure(figsize=(30,20))
# plt.errorbar(logR_p, gamT_p, yerr=gam_sig_p, capsize = 3, label='gamma_T')
# plt.boxplot(res[:,2,:], positions=logR_p, widths=0.2)

plt.errorbar(R_p, gamT_p, yerr=gam_sig_p, capsize = 3, label=r'Data')
plt.plot(R_p, stacked_gammaT_theo/len(sdss_cluster_cut['M']), label=r'Theoretical NFW profile')
plt.xscale('log')
plt.hlines(y=0, xmin=plt.xlim()[0], xmax=plt.xlim()[1], linestyles='dashed')
plt.xlim(plt.xlim()[0]+0.13,plt.xlim()[1]-17)
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\gamma_{t}$')
plt.legend()
plt.title('Stacked cluster profiles')
plt.savefig(plot_dir + 'all_clusters_profile.png')

In [ ]:
R_m, logR_m, gamT_m, gamX_m, gam_sig_m = gamma_T_tc(sdss_cluster_cut['ra'], sdss_cluster_cut['dec'], ra_ngmix, dec_ngmix, g_corr_ngmix[0]-c1_ngmix, g_corr_ngmix[1]-c2_ngmix, w_ngmix)

In [ ]:
mean_z_clust = np.mean(sdss_cluster_cut['z'])
m_est, fitted_profile = get_theo_mass(mean_z_clust, R_m, gamT_m, gam_sig_m, 0.98, meanbin_z=z, nz=nz)

In [ ]:
print("Cluster mean redshift : {}".format(mean_z_clust))
print("Estimated mass : {:.3E}".format(m_est))
print('"True" mass : {:.3E}'.format(np.mean(sdss_cluster_cut['M'])*0.7))
stat_file.write("Cluster mean redshift : {}\n".format(mean_z_clust))
stat_file.write("Estimated mass : {:.3E}\n".format(m_est))
stat_file.write('"True" mass : {:.3E}\n'.format(np.mean(sdss_cluster_cut['M'])*0.7))

In [ ]:
plt.figure(figsize=(30,20))
plt.errorbar(R_m, gamT_m, yerr=gam_sig_m, label=r'Data')
plt.plot(R_m, fitted_profile, label=r'Best fit NFW profile')
plt.hlines(y=0, xmin=np.min(R_m), xmax=np.max(R_m), linestyles='dashed')
plt.xscale('log')
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\gamma_{t}$')
plt.legend()
plt.title('Mass recovery (best fit)')

### Mass maps

#### Full mass map

In [ ]:
#from lenspack.utils import bin2d
#from lenspack.geometry.projections.gnom import radec2xy

In [ ]:
xx, yy = radec2xy(np.mean(ra_ngmix), np.mean(dec_ngmix), ra_ngmix, dec_ngmix)

In [ ]:
help(radec2xy)

In [ ]:
#e1map, e2map = bin2d(correct_ra2(ra[m_test], dec[m_test]), dec[m_test], v=(g_corr[0][m_test]-c1, g_corr[1][m_test]-c2), w=w[m_test], npix=2048, verbose=True)

In [ ]:
e1map, e2map = bin2d(xx, yy, v=(g_corr_ngmix[0]-c1_ngmix, g_corr_ngmix[1]-c2_ngmix), w=w_ngmix, npix=4096, verbose=True)

In [ ]:
kappaE, kappaB = ks93(e1map, -e2map)

In [ ]:
from scipy.ndimage.filters import gaussian_filter as gf

In [ ]:
xx_cluster, yy_cluster = radec2xy(np.mean(ra_ngmix), np.mean(dec_ngmix), sdss_cluster_cut['ra'], sdss_cluster_cut['dec'])
plt.figure(figsize=(20,20))
plt.imshow(gf(kappaE, 16))
vminE, vmaxE = plt.gci().get_clim()
x_cluster = (xx_cluster - -0.24970972516329504) / 0.00011016574322916806
y_cluster = (yy_cluster - -0.12938850035423424) / 7.312034604994855e-05
dy = 0.02
plt.plot(x_cluster, y_cluster, 'k+')
for i in range(len(sdss_cluster_cut['z'])):
    plt.text(x_cluster[i], y_cluster[i] + dy * (plt.ylim()[0] - plt.ylim()[1]), round(sdss_cluster_cut['z'][i],3), color='k', fontsize=10, ha='center', va='center')
    plt.text(x_cluster[i], y_cluster[i] - dy * (plt.ylim()[0] - plt.ylim()[1]), round(sdss_cluster_cut['M'][i]/1e14, 2), color='k', fontsize=10, ha='center', va='center')
plt.colorbar()

loc, labels = plt.xticks()
plt.xticks(loc[1:-1], labels=np.round(np.arange(ra_ngmix.min(), ra_ngmix.max(), step=(ra_ngmix.max()-ra_ngmix.min())/len(loc[1:-1])), 1))
loc, labels = plt.yticks()
plt.yticks(loc[1:-1], labels=np.round(np.arange(dec_ngmix.min(), dec_ngmix.max(), step=(dec_ngmix.max()-dec_ngmix.min())/len(loc[1:-1])), 1))
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()
plt.xlabel('Ra (deg)')
plt.ylabel('Dec (deg)')
plt.title('$\kappa_{E}$')
plt.savefig(plot_dir + 'mass_map_planck_cluster_E.png')

In [ ]:
xx_cluster, yy_cluster = radec2xy(np.mean(ra_ngmix), np.mean(dec_ngmix), sdss_cluster_cut['ra'], sdss_cluster_cut['dec'])
plt.figure(figsize=(20,20))
plt.imshow(gf(kappaB, 16), vmin=vminE, vmax=vmaxE)
x_cluster = (xx_cluster - -0.24970972516329504) / 0.00011016574322916806
y_cluster = (yy_cluster - -0.12938850035423424) / 7.312034604994855e-05
dy = 0.02
plt.plot(x_cluster, y_cluster, 'k+')
for i in range(len(sdss_cluster_cut['z'])):
    plt.text(x_cluster[i], y_cluster[i] + dy * (plt.ylim()[0] - plt.ylim()[1]), round(sdss_cluster_cut['z'][i],3), color='k', fontsize=10, ha='center', va='center')
    plt.text(x_cluster[i], y_cluster[i] - dy * (plt.ylim()[0] - plt.ylim()[1]), round(sdss_cluster_cut['M'][i]/1e14, 2), color='k', fontsize=10, ha='center', va='center')
plt.colorbar()

loc, labels = plt.xticks()
plt.xticks(loc[1:-1], labels=np.round(np.arange(ra_ngmix.min(), ra_ngmix.max(), step=(ra_ngmix.max()-ra_ngmix.min())/len(loc[1:-1])), 1))
loc, labels = plt.yticks()
plt.yticks(loc[1:-1], labels=np.round(np.arange(dec_ngmix.min(), dec_ngmix.max(), step=(dec_ngmix.max()-dec_ngmix.min())/len(loc[1:-1])), 1))
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()
plt.xlabel('Ra (deg)')
plt.ylabel('Dec (deg)')
plt.title('$\kappa_{B}$')
plt.savefig(plot_dir + 'mass_map_planck_cluster_B.png')

#### Stacked mass map

In [ ]:
radius = 5

In [ ]:
res_stack_mm = stack_mm3(ra_ngmix, dec_ngmix, g_corr_ngmix[0]-c1_ngmix, g_corr_ngmix[1]-c2_ngmix, w_ngmix, sdss_cluster_cut['ra'], sdss_cluster_cut['dec'], sdss_cluster_cut['z'], radius=radius, n_match=1000000)

In [ ]:
len(res_stack_mm[0])

In [ ]:
plt.figure(figsize=(20,20))
plt.hexbin(res_stack_mm[0], res_stack_mm[1], gridsize=100, cmap='gist_stern')
cbar = plt.colorbar()
cbar.set_label('Number count', rotation=270)
plt.title('Density plot')

In [ ]:
#e1map, e2map = bin2d(correct_ra2(res_stack_mm[0], res_stack_mm[1]), res_stack_mm[1], v=(res_stack_mm[2], res_stack_mm[2]), w=res_stack_mm[4], npix=1024, verbose=True)
e1map_stack, e2map_stack = bin2d(res_stack_mm[0], res_stack_mm[1], v=(res_stack_mm[2], -res_stack_mm[3]), w=res_stack_mm[4], npix=2048, verbose=True)

In [ ]:
kappaE_stack, kappaB_stack = ks93(e1map_stack, e2map_stack)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(gf(kappaE_stack, 32))
vminEs, vmaxEs = plt.gci().get_clim()
plt.plot(1023, 1023, '+')
n_ticks = 4
plt.xticks(np.arange(0, 2048+2048/n_ticks, step=2048/n_ticks), labels=np.round(np.arange(-radius, radius+radius*2/n_ticks, step=radius*2/n_ticks), 1))
plt.yticks(np.arange(0, 2048+2048/n_ticks, step=2048/n_ticks), labels=np.round(np.arange(-radius, radius+radius*2/n_ticks, step=radius*2/n_ticks), 1))
cbar = plt.colorbar()
cbar.set_label('$\kappa_{E}$')
plt.xlabel('Mpc')
plt.ylabel('Mpc')
plt.title(r'$\kappa_{E}$ map')
plt.savefig(plot_dir + 'cluster_mass_map_E.png')

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(gf(kappaB_stack, 32), vmin=vminEs, vmax=vmaxEs)
plt.plot(1023, 1023, '+')
n_ticks = 4
plt.xticks(np.arange(0, 2048+2048/n_ticks, step=2048/n_ticks), labels=np.round(np.arange(-radius, radius+radius*2/n_ticks, step=radius*2/n_ticks), 1))
plt.yticks(np.arange(0, 2048+2048/n_ticks, step=2048/n_ticks), labels=np.round(np.arange(-radius, radius+radius*2/n_ticks, step=radius*2/n_ticks), 1))
cbar = plt.colorbar()
cbar.set_label('$\kappa_{B}$')
plt.xlabel('Mpc')
plt.ylabel('Mpc')
plt.title(r'$\kappa_{B}$ map')
plt.savefig(plot_dir + 'cluster_mass_map_B.png')

#### Close stat file

In [ ]:
stat_file.close()